这个notebook的目标是，处理新冠数据2020-2021两整年，然后合并到costar数据中，也就是分析年份扩张到2018-2021一共四年。

# 导入新冠数据，提取佛州相关数据。

In [2]:
import numpy as np
import pandas as pd




In [2]:
covid=pd.read_csv('/Users/yoga/Documents/Projects/1_covid19_cre/72_update regression analysis 20220122/1_us-counties_covid20220122.csv')

covid_fl=covid.loc[covid['state']=='Florida']
covid_fl


,date,county,state,fips,cases,deaths
411,2020-03-01,Hillsborough,Florida,12057.0,1,0.0
412,2020-03-01,Manatee,Florida,12081.0,1,0.0
440,2020-03-02,Hillsborough,Florida,12057.0,1,0.0
441,2020-03-02,Manatee,Florida,12081.0,1,0.0
474,2020-03-03,Hillsborough,Florida,12057.0,2,0.0
...,...,...,...,...,...,...
2138790,2022-01-22,Unknown,Florida,NaN,22916,605.0
2138791,2022-01-22,Volusia,Florida,12127.0,102609,1659.0
2138792,2022-01-22,Wakulla,Florida,12129.0,8520,108.0
2138793,2022-01-22,Walton,Florida,12131.0,14760,177.0


## calculate the aggregate number of Florida case and death in each quarter

In [3]:
time=['2020-03-31','2020-06-30','2020-09-30','2020-12-31','2021-03-31','2021-06-30','2021-09-30','2021-12-31']
covid_fl_qtr=covid_fl.loc[covid_fl['date'].isin(time)]

covid_fl_qtr


,date,county,state,fips,cases,deaths
21976,2020-03-31,Alachua,Florida,12001.0,87,0.0
21977,2020-03-31,Baker,Florida,12003.0,9,0.0
21978,2020-03-31,Bay,Florida,12005.0,8,0.0
21979,2020-03-31,Bradford,Florida,12007.0,2,0.0
21980,2020-03-31,Brevard,Florida,12009.0,37,0.0
...,...,...,...,...,...,...
2067246,2021-12-31,Unknown,Florida,NaN,5,162.0
2067247,2021-12-31,Volusia,Florida,12127.0,82954,1647.0
2067248,2021-12-31,Wakulla,Florida,12129.0,6803,106.0
2067249,2021-12-31,Walton,Florida,12131.0,12464,173.0


In [10]:
#create two dictionary of date to death and case
time=['2020-03-31','2020-06-30','2020-09-30','2020-12-31','2021-03-31','2021-06-30','2021-09-30','2021-12-31']
#this is the aggregate number
date2case={}
date2death={}

for i in range(len(time)):
    time1=time[i]
    covid_one_qtr=covid_fl_qtr[covid_fl_qtr['date']==time1]
    case_qtr=covid_one_qtr['cases'].sum()
    death_qtr=covid_one_qtr['deaths'].sum()
    date2case[time1]=case_qtr
    date2death[time1]=death_qtr
    
    

In [26]:
pd2date2case=pd.DataFrame.from_dict(date2case.items())
pd2date2case = pd2date2case.rename(columns={0: 'date', 1: 'aggregate_case'})

pd2date2case

,date,aggregate_case
0,2020-03-31,6742
1,2020-06-30,152426
2,2020-09-30,706508
3,2020-12-31,1323307
4,2021-03-31,2057727
5,2021-06-30,2329484
6,2021-09-30,3570752
7,2021-12-31,4166392


In [30]:
#calculate current number case
pd2date2case['current_case']=''
for i in range(len(pd2date2case)):
    if i ==0:
        pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]
    if i >0:
        pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]-pd2date2case['aggregate_case'][i-1]
        

<ipython-input-30-7bc0ac5965b3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]
<ipython-input-30-7bc0ac5965b3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]-pd2date2case['aggregate_case'][i-1]


In [31]:
pd2date2case

,date,aggregate_case,current_case
0,2020-03-31,6742,6742
1,2020-06-30,152426,145684
2,2020-09-30,706508,554082
3,2020-12-31,1323307,616799
4,2021-03-31,2057727,734420
5,2021-06-30,2329484,271757
6,2021-09-30,3570752,1241268
7,2021-12-31,4166392,595640


In [24]:
pd2date2death=pd.DataFrame.from_dict(date2death.items())
pd2date2death = pd2date2death.rename(columns={0: 'date', 1: 'aggregate_death'})

In [25]:
pd2date2death

,date,aggregate_death
0,2020-03-31,85.0
1,2020-06-30,3504.0
2,2020-09-30,14316.0
3,2020-12-31,21672.0
4,2021-03-31,33424.0
5,2021-06-30,37877.0
6,2021-09-30,55009.0
7,2021-12-31,62504.0


In [27]:
#calculate current number death
pd2date2death['current_death']=''
for i in range(len(pd2date2death)):
    if i ==0:
        pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]
    if i >0:
        pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]-pd2date2death['aggregate_death'][i-1]
        





<ipython-input-27-2d7ab747961f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]
<ipython-input-27-2d7ab747961f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]-pd2date2death['aggregate_death'][i-1]


In [28]:
pd2date2death

,date,aggregate_death,current_death
0,2020-03-31,85.0,85.0
1,2020-06-30,3504.0,3419.0
2,2020-09-30,14316.0,10812.0
3,2020-12-31,21672.0,7356.0
4,2021-03-31,33424.0,11752.0
5,2021-06-30,37877.0,4453.0
6,2021-09-30,55009.0,17132.0
7,2021-12-31,62504.0,7495.0


In [33]:
df_concat = pd.concat([pd2date2death, pd2date2case], axis=1)
df_concat

,date,aggregate_death,current_death,date,aggregate_case,current_case
0,2020-03-31,85.0,85.0,2020-03-31,6742,6742
1,2020-06-30,3504.0,3419.0,2020-06-30,152426,145684
2,2020-09-30,14316.0,10812.0,2020-09-30,706508,554082
3,2020-12-31,21672.0,7356.0,2020-12-31,1323307,616799
4,2021-03-31,33424.0,11752.0,2021-03-31,2057727,734420
5,2021-06-30,37877.0,4453.0,2021-06-30,2329484,271757
6,2021-09-30,55009.0,17132.0,2021-09-30,3570752,1241268
7,2021-12-31,62504.0,7495.0,2021-12-31,4166392,595640


In [35]:
df_concat2=df_concat[['date','current_case','current_death']]

In [40]:
df_concat2.columns=['date', 'date2', 'current_case', 'current_death', 'quarter']

In [41]:
df_concat2

,date,date2,current_case,current_death,quarter
0,2020-03-31,2020-03-31,6742,85.0,
1,2020-06-30,2020-06-30,145684,3419.0,
2,2020-09-30,2020-09-30,554082,10812.0,
3,2020-12-31,2020-12-31,616799,7356.0,
4,2021-03-31,2021-03-31,734420,11752.0,
5,2021-06-30,2021-06-30,271757,4453.0,
6,2021-09-30,2021-09-30,1241268,17132.0,
7,2021-12-31,2021-12-31,595640,7495.0,


In [42]:
df_concat2['quarter']=''
df_concat2['quarter'][df_concat2['date']=='2020-03-31']='2020 Q1'
df_concat2['quarter'][df_concat2['date']=='2020-06-30']='2020 Q2'
df_concat2['quarter'][df_concat2['date']=='2020-09-30']='2020 Q3'
df_concat2['quarter'][df_concat2['date']=='2020-12-31']='2020 Q4'

df_concat2['quarter'][df_concat2['date']=='2021-03-31']='2021 Q1'
df_concat2['quarter'][df_concat2['date']=='2021-06-30']='2021 Q2'
df_concat2['quarter'][df_concat2['date']=='2021-09-30']='2021 Q3'
df_concat2['quarter'][df_concat2['date']=='2021-12-31']='2021 Q4'


<ipython-input-42-2bd9bcda102b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat2['quarter']=''
/Users/yoga/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [43]:
df_concat2

,date,date2,current_case,current_death,quarter
0,2020-03-31,2020-03-31,6742,85.0,2020 Q1
1,2020-06-30,2020-06-30,145684,3419.0,2020 Q2
2,2020-09-30,2020-09-30,554082,10812.0,2020 Q3
3,2020-12-31,2020-12-31,616799,7356.0,2020 Q4
4,2021-03-31,2021-03-31,734420,11752.0,2021 Q1
5,2021-06-30,2021-06-30,271757,4453.0,2021 Q2
6,2021-09-30,2021-09-30,1241268,17132.0,2021 Q3
7,2021-12-31,2021-12-31,595640,7495.0,2021 Q4


In [44]:
#df_concat2.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/10_florida_covid_number.csv')

## calculate the aggregate number of united states 

In [47]:
time=['2020-03-31','2020-06-30','2020-09-30','2020-12-31','2021-03-31','2021-06-30','2021-09-30','2021-12-31']
covid_us=covid.loc[covid['date'].isin(time)]
covid_us

,date,county,state,fips,cases,deaths
21727,2020-03-31,Autauga,Alabama,1001.0,7,0.0
21728,2020-03-31,Baldwin,Alabama,1003.0,19,0.0
21729,2020-03-31,Bibb,Alabama,1007.0,3,0.0
21730,2020-03-31,Blount,Alabama,1009.0,5,0.0
21731,2020-03-31,Bullock,Alabama,1011.0,3,0.0
...,...,...,...,...,...,...
2070105,2021-12-31,Sweetwater,Wyoming,56037.0,8320,107.0
2070106,2021-12-31,Teton,Wyoming,56039.0,5950,14.0
2070107,2021-12-31,Uinta,Wyoming,56041.0,4154,31.0
2070108,2021-12-31,Washakie,Wyoming,56043.0,1879,37.0


In [48]:
#create two dictionary of date to death and case
time=['2020-03-31','2020-06-30','2020-09-30','2020-12-31','2021-03-31','2021-06-30','2021-09-30','2021-12-31']
#this is the aggregate number for the united states
date2case={}
date2death={}

for i in range(len(time)):
    time1=time[i]
    covid_one_qtr=covid_us[covid_us['date']==time1]
    case_qtr=covid_one_qtr['cases'].sum()
    death_qtr=covid_one_qtr['deaths'].sum()
    date2case[time1]=case_qtr
    date2death[time1]=death_qtr
    

In [50]:
pd2date2case=pd.DataFrame.from_dict(date2case.items())
pd2date2case = pd2date2case.rename(columns={0: 'date', 1: 'aggregate_case'})

pd2date2case

,date,aggregate_case
0,2020-03-31,188461
1,2020-06-30,2653321
2,2020-09-30,7262735
3,2020-12-31,20024801
4,2021-03-31,30478886
5,2021-06-30,33639764
6,2021-09-30,43458032
7,2021-12-31,54689357


In [51]:
#calculate current number case
pd2date2case['current_case']=''
for i in range(len(pd2date2case)):
    if i ==0:
        pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]
    if i >0:
        pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]-pd2date2case['aggregate_case'][i-1]
        

<ipython-input-51-7bc0ac5965b3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]
<ipython-input-51-7bc0ac5965b3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2case['current_case'][i]=pd2date2case['aggregate_case'][i]-pd2date2case['aggregate_case'][i-1]


,date,aggregate_case,current_case
0,2020-03-31,188461,188461
1,2020-06-30,2653321,2464860
2,2020-09-30,7262735,4609414
3,2020-12-31,20024801,12762066
4,2021-03-31,30478886,10454085
5,2021-06-30,33639764,3160878
6,2021-09-30,43458032,9818268
7,2021-12-31,54689357,11231325


In [53]:
pd2date2death=pd.DataFrame.from_dict(date2death.items())
pd2date2death = pd2date2death.rename(columns={0: 'date', 1: 'aggregate_death'})


#calculate current number death
pd2date2death['current_death']=''
for i in range(len(pd2date2death)):
    if i ==0:
        pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]
    if i >0:
        pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]-pd2date2death['aggregate_death'][i-1]




<ipython-input-53-365c423019e5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]
<ipython-input-53-365c423019e5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd2date2death['current_death'][i]=pd2date2death['aggregate_death'][i]-pd2date2death['aggregate_death'][i-1]


In [60]:
df_concat = pd.concat([pd2date2death, pd2date2case], axis=1)
#df_concat

df_concat2=df_concat[['date','current_case','current_death']]
#df_concat2
df_concat2.columns=['date', 'date2', 'current_case', 'current_death']
df_concat2

,date,date2,current_case,current_death
0,2020-03-31,2020-03-31,188461,4289.0
1,2020-06-30,2020-06-30,2464860,123173.0
2,2020-09-30,2020-09-30,4609414,79390.0
3,2020-12-31,2020-12-31,12762066,139198.0
4,2021-03-31,2021-03-31,10454085,205611.0
5,2021-06-30,2021-06-30,3160878,52799.0
6,2021-09-30,2021-09-30,9818268,93676.0
7,2021-12-31,2021-12-31,11231325,126200.0


In [61]:
df_concat2['quarter']=''
df_concat2['quarter'][df_concat2['date']=='2020-03-31']='2020 Q1'
df_concat2['quarter'][df_concat2['date']=='2020-06-30']='2020 Q2'
df_concat2['quarter'][df_concat2['date']=='2020-09-30']='2020 Q3'
df_concat2['quarter'][df_concat2['date']=='2020-12-31']='2020 Q4'

df_concat2['quarter'][df_concat2['date']=='2021-03-31']='2021 Q1'
df_concat2['quarter'][df_concat2['date']=='2021-06-30']='2021 Q2'
df_concat2['quarter'][df_concat2['date']=='2021-09-30']='2021 Q3'
df_concat2['quarter'][df_concat2['date']=='2021-12-31']='2021 Q4'


<ipython-input-61-2bd9bcda102b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat2['quarter']=''
/Users/yoga/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [64]:
df_concat2.columns=['date', 'date2', 'US_current_case', 'US_current_death', 'quarter']


In [66]:
#df_concat2.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/11_US_covid_number.csv')

In [20]:


covid_fl_qtr['quarter']=''
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2020-03-31']='2020 Q1'
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2020-06-30']='2020 Q2'
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2020-09-30']='2020 Q3'
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2020-12-31']='2020 Q4'

covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2021-03-31']='2021 Q1'
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2021-06-30']='2021 Q2'
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2021-09-30']='2021 Q3'
covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2021-12-31']='2021 Q4'



<ipython-input-20-6bf9af656386>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_fl_qtr['quarter']=''
<ipython-input-20-6bf9af656386>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_fl_qtr['quarter'][covid_fl_qtr['date']=='2020-03-31']='2020 Q1'
/Users/yoga/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [22]:
covid_fl_qtr

,date,county,state,fips,cases,deaths,quarter
21976,2020-03-31,Alachua,Florida,12001.0,87,0.0,2020 Q1
21977,2020-03-31,Baker,Florida,12003.0,9,0.0,2020 Q1
21978,2020-03-31,Bay,Florida,12005.0,8,0.0,2020 Q1
21979,2020-03-31,Bradford,Florida,12007.0,2,0.0,2020 Q1
21980,2020-03-31,Brevard,Florida,12009.0,37,0.0,2020 Q1
...,...,...,...,...,...,...,...
2067246,2021-12-31,Unknown,Florida,NaN,5,162.0,2021 Q4
2067247,2021-12-31,Volusia,Florida,12127.0,82954,1647.0,2021 Q4
2067248,2021-12-31,Wakulla,Florida,12129.0,6803,106.0,2021 Q4
2067249,2021-12-31,Walton,Florida,12131.0,12464,173.0,2021 Q4


In [23]:
#这里缺少三个MSA,如下：33124  Miami - FL48424  Palm Beach - FL；22744 Fort Lauderdale - FL

cbsa2coun=pd.read_excel('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/4-5 cbsa 2 county name.xlsx')
cbsa2coun


,CBSAFP,CBSA NAME,County NAME
0,23540,"Gainesville, FL",Levy
1,23540,"Gainesville, FL",Alachua
2,23540,"Gainesville, FL",Gilchrist
3,18880,"Crestview-Fort Walton Beach-Destin, FL",Walton
4,18880,"Crestview-Fort Walton Beach-Destin, FL",Okaloosa
5,19660,"Deltona-Daytona Beach-Ormond Beach, FL",Flagler
6,19660,"Deltona-Daytona Beach-Ormond Beach, FL",Volusia
7,15980,"Cape Coral-Fort Myers, FL",Lee
8,26140,"Homosassa Springs, FL",Citrus
9,27260,"Jacksonville, FL",Clay


In [24]:
len(cbsa2coun['CBSA NAME'].unique())

24

In [25]:
#try to merge
covid2_merge=pd.merge(left=covid_fl_qtr,right=cbsa2coun,left_on='county',right_on='County NAME',how='left')


In [26]:
covid2_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531 entries, 0 to 530
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         531 non-null    object 
 1   county       531 non-null    object 
 2   state        531 non-null    object 
 3   fips         524 non-null    float64
 4   cases        531 non-null    int64  
 5   deaths       531 non-null    float64
 6   quarter      531 non-null    object 
 7   CBSAFP       350 non-null    float64
 8   CBSA NAME    350 non-null    object 
 9   County NAME  350 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 45.6+ KB


In [27]:
len(covid2_merge['CBSAFP'].unique()) #多了一个nan

25

In [28]:
len(cbsa2coun['County NAME'].unique())


44

In [29]:
covid2_merge2=covid2_merge.copy()
covid2_merge2=covid2_merge2.dropna()

In [30]:
len(covid2_merge2['County NAME'].unique())

44

In [31]:
covid2_merge2

,date,county,state,fips,cases,deaths,quarter,CBSAFP,CBSA NAME,County NAME
0,2020-03-31,Alachua,Florida,12001.0,87,0.0,2020 Q1,23540.0,"Gainesville, FL",Alachua
1,2020-03-31,Baker,Florida,12003.0,9,0.0,2020 Q1,27260.0,"Jacksonville, FL",Baker
2,2020-03-31,Bay,Florida,12005.0,8,0.0,2020 Q1,37460.0,"Panama City, FL",Bay
4,2020-03-31,Brevard,Florida,12009.0,37,0.0,2020 Q1,37340.0,"Palm Bay-Melbourne-Titusville, FL",Brevard
5,2020-03-31,Broward,Florida,12011.0,1219,12.0,2020 Q1,22744.0,"Fort Lauderdale, FL",Broward
...,...,...,...,...,...,...,...,...,...,...
521,2021-12-31,St. Lucie,Florida,12111.0,52135,1047.0,2021 Q4,38940.0,"Port St. Lucie, FL",St. Lucie
522,2021-12-31,Sumter,Florida,12119.0,15405,475.0,2021 Q4,45540.0,"The Villages, FL",Sumter
527,2021-12-31,Volusia,Florida,12127.0,82954,1647.0,2021 Q4,19660.0,"Deltona-Daytona Beach-Ormond Beach, FL",Volusia
528,2021-12-31,Wakulla,Florida,12129.0,6803,106.0,2021 Q4,45220.0,"Tallahassee, FL",Wakulla


In [32]:
covid2_merge2['quarter_CBSAFP']=covid2_merge2['quarter'].astype(str)+"&"+covid2_merge2['CBSAFP'].astype(int).astype(str)
covid2_merge2['quarter_CBSAFP']

0      2020 Q1&23540
1      2020 Q1&27260
2      2020 Q1&37460
4      2020 Q1&37340
5      2020 Q1&22744
           ...      
521    2021 Q4&38940
522    2021 Q4&45540
527    2021 Q4&19660
528    2021 Q4&45220
529    2021 Q4&18880
Name: quarter_CBSAFP, Length: 350, dtype: object

In [33]:


covid2_merge2[covid2_merge2['quarter_CBSAFP']=='2021 Q4&38940']

,date,county,state,fips,cases,deaths,quarter,CBSAFP,CBSA NAME,County NAME,quarter_CBSAFP
504,2021-12-31,Martin,Florida,12085.0,22335,523.0,2021 Q4,38940.0,"Port St. Lucie, FL",Martin,2021 Q4&38940
521,2021-12-31,St. Lucie,Florida,12111.0,52135,1047.0,2021 Q4,38940.0,"Port St. Lucie, FL",St. Lucie,2021 Q4&38940


In [34]:
covid2_merge2.loc[i]['county']

'Alachua'

In [35]:
#create a dict to calcualte the cases and death.
quarter_CBSAFP2case={}
quarter_CBSAFP2death={}

for i in covid2_merge2.index:
    covid2_merge2i=covid2_merge2.loc[i]['quarter_CBSAFP']
    
    if covid2_merge2i in quarter_CBSAFP2case.keys():
        quarter_CBSAFP2case[covid2_merge2i]=quarter_CBSAFP2case[covid2_merge2i]+covid2_merge2.loc[i]['cases']
        quarter_CBSAFP2death[covid2_merge2i]=quarter_CBSAFP2death[covid2_merge2i]+covid2_merge2.loc[i]['deaths']   
    else:      
        quarter_CBSAFP2case[covid2_merge2i]=covid2_merge2.loc[i]['cases']
        quarter_CBSAFP2death[covid2_merge2i]=covid2_merge2.loc[i]['deaths']



In [36]:

quarter_CBSAFP2case['2021 Q4&38940']

74470

In [37]:
22335+52135

74470

In [50]:
pd_quarter_CBSAFP2case=pd.DataFrame(quarter_CBSAFP2case.items())
pd_quarter_CBSAFP2death=pd.DataFrame(quarter_CBSAFP2death.items())

pd_quarter_CBSAFP2death = pd_quarter_CBSAFP2death.rename({0:'quarter_cbsa', 1:'aggregate_death'}, axis='columns')
pd_quarter_CBSAFP2death

pd_quarter_CBSAFP2case = pd_quarter_CBSAFP2case.rename({0:'quarter_cbsa', 1:'aggregate_case'}, axis='columns')
pd_quarter_CBSAFP2case

pd_quarter_CBSAFP2case_death=pd_quarter_CBSAFP2case.merge(pd_quarter_CBSAFP2death,left_on='quarter_cbsa',right_on='quarter_cbsa',how='left')

#pd_quarter_CBSAFP2case_death.to_csv('20_aggregate_death_case_covid_2020_2021.csv',index=0)





In [52]:
pd_quarter_CBSAFP2case_death

,quarter_cbsa,aggregate_case,aggregate_death
0,2020 Q1&23540,89,0.0
1,2020 Q1&27260,346,11.0
2,2020 Q1&37460,8,0.0
3,2020 Q1&37340,37,0.0
4,2020 Q1&22744,1219,12.0
...,...,...,...
187,2021 Q4&33124,847746,9260.0
188,2021 Q4&18880,48621,754.0
189,2021 Q4&48424,268242,4322.0
190,2021 Q4&29460,140400,2531.0


## 把累计数字变成当期数字


In [90]:
cbsa2deaths_current=dict()
cbsa2cases_current=dict()

quarter=['2020 Q1','2020 Q2','2020 Q3','2020 Q4','2021 Q1','2021 Q2','2021 Q3','2021 Q4']

for i in pd_quarter_CBSAFP2case_death.index:
    cbsa=pd_quarter_CBSAFP2case_death['quarter_cbsa'][i]
    cbsa_period=cbsa.split('&')[0]
    cbsa_code=cbsa.split('&')[1]
    if cbsa_period=='2020 Q1':
        cbsa2deaths_current[cbsa]=pd_quarter_CBSAFP2case_death['aggregate_death'][i]
        cbsa2cases_current[cbsa]=pd_quarter_CBSAFP2case_death['aggregate_case'][i]
    else:
        if cbsa_period not in quarter:
            print('Could not find ', cbsa_period)
            break
        cur_indx = quarter.index(cbsa_period)
        pre_cbsa_period = quarter[cur_indx-1]
        pre_cbsa_code=cbsa_code
        pre_cbsa=pre_cbsa_period+'&'+str(pre_cbsa_code)
        pre_case=pd_quarter_CBSAFP2case_death[pd_quarter_CBSAFP2case_death['quarter_cbsa']==pre_cbsa]['aggregate_case'].values[0]
        pre_death=pd_quarter_CBSAFP2case_death[pd_quarter_CBSAFP2case_death['quarter_cbsa']==pre_cbsa]['aggregate_death'].values[0]
       
        cbsa2deaths_current[cbsa]=pd_quarter_CBSAFP2case_death['aggregate_death'][i]-pre_death
        cbsa2cases_current[cbsa]=pd_quarter_CBSAFP2case_death['aggregate_case'][i]-pre_case
        
        
        
        
        #print(pre_cbsa_period, '->', cbsa_period)
       
        
        
        
        
    
    
    

In [92]:
pd_quarter_CBSAFP2case_current=pd.DataFrame(cbsa2cases_current.items())
pd_quarter_CBSAFP2death_current=pd.DataFrame(cbsa2deaths_current.items())

pd_quarter_CBSAFP2death_current = pd_quarter_CBSAFP2death_current.rename({0:'quarter_cbsa', 1:'current_death'}, axis='columns')
pd_quarter_CBSAFP2death

pd_quarter_CBSAFP2case_current = pd_quarter_CBSAFP2case_current.rename({0:'quarter_cbsa', 1:'current_case'}, axis='columns')
pd_quarter_CBSAFP2case

pd_quarter_CBSAFP2case_death_current=pd_quarter_CBSAFP2case_current.merge(pd_quarter_CBSAFP2death_current,left_on='quarter_cbsa',right_on='quarter_cbsa',how='left')


In [98]:
#pd_quarter_CBSAFP2case_death_current.to_csv('21_current_death_case_covid_2020_2021.csv',index=0)


#pd_quarter_CBSAFP2case_death.to_csv('20_aggregate_death_case_covid_2020_2021.csv',index=0)

In [94]:
pd_quarter_CBSAFP2case_death

,quarter_cbsa,aggregate_case,aggregate_death
0,2020 Q1&23540,89,0.0
1,2020 Q1&27260,346,11.0
2,2020 Q1&37460,8,0.0
3,2020 Q1&37340,37,0.0
4,2020 Q1&22744,1219,12.0
...,...,...,...
187,2021 Q4&33124,847746,9260.0
188,2021 Q4&18880,48621,754.0
189,2021 Q4&48424,268242,4322.0
190,2021 Q4&29460,140400,2531.0


In [95]:
pd_quarter_CBSAFP2case_death[pd_quarter_CBSAFP2case_death['quarter_cbsa']=='2021 Q3&45540']

,quarter_cbsa,aggregate_case,aggregate_death
167,2021 Q3&45540,13990,428.0


# 导入costar data, 进行合并 新冠的数据

In [3]:
retail=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/72_update regression analysis 20220122/Retail_01-18-2022.xlsx')
office=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/72_update regression analysis 20220122/Office_01-18-2022.xlsx')
industry=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/72_update regression analysis 20220122/Industrial_01-18-2022.xlsx')



In [54]:
#df[df['ids'].str.contains("ball")]
retail2=retail[(retail['Period'].str.contains('2018 Q'))|(retail['Period'].str.contains('2019 Q'))|(retail['Period'].str.contains('2020 Q'))|(retail['Period'].str.contains('2021 Q'))]

office2=office[(office['Period'].str.contains('2018 Q'))|(office['Period'].str.contains('2019 Q'))|(office['Period'].str.contains('2020 Q'))|(office['Period'].str.contains('2021 Q'))]

industry2=industry[(industry['Period'].str.contains('2018 Q'))|(industry['Period'].str.contains('2019 Q'))|(industry['Period'].str.contains('2020 Q'))|(industry['Period'].str.contains('2021 Q'))]



In [55]:
industry2

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Sold Building SF,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF
144,Industrial,2018 Q1,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,227031.0,202634.593750,0.121489,16597775.0,82.98,4.0,54000.0,0.029080,0.026570,636770.0
145,Industrial,2018 Q2,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,623631.0,203406.906250,0.123515,41943208.0,71.42,3.0,36000.0,0.032372,0.028699,688304.0
146,Industrial,2018 Q3,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,171463.0,204901.390625,0.124135,5165600.0,44.95,4.0,37160.0,0.031292,0.030832,739480.0
147,Industrial,2018 Q4,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,192826.0,205294.500000,0.125630,5367000.0,46.90,5.0,93120.0,0.043182,0.043182,1035788.0
148,Industrial,2019 Q1,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,76136.0,205313.906250,0.125999,1646000.0,72.46,3.0,124960.0,0.046645,0.043040,1033872.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3858,Industrial,2020 Q4,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,NaN,31842.000000,0.138026,0.0,NaN,7.0,81600.0,0.021936,0.021936,34530.0
3859,Industrial,2021 Q1,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,13920.0,31496.189453,0.136862,1400000.0,100.57,7.0,81600.0,0.013023,0.013023,20500.0
3860,Industrial,2021 Q2,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,72792.0,31540.269531,0.142301,5357195.0,73.60,7.0,81600.0,0.048391,0.048391,76172.0
3861,Industrial,2021 Q3,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,23266.0,31965.048828,0.156650,3750000.0,161.18,2.0,21600.0,0.054643,0.054643,89292.0


In [56]:
#generate the key word for merge.

retail2['quarter_CBSAFP']=retail2['Period'].astype(str)+"&"+retail2['CBSA Code'].astype(str)

office2['quarter_CBSAFP']=office2['Period'].astype(str)+"&"+office2['CBSA Code'].astype(str)

industry2['quarter_CBSAFP']=industry2['Period'].astype(str)+"&"+industry2['CBSA Code'].astype(str)



<ipython-input-56-c4f0b6e586eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retail2['quarter_CBSAFP']=retail2['Period'].astype(str)+"&"+retail2['CBSA Code'].astype(str)
<ipython-input-56-c4f0b6e586eb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  office2['quarter_CBSAFP']=office2['Period'].astype(str)+"&"+office2['CBSA Code'].astype(str)
<ipython-input-56-c4f0b6e586eb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [57]:
industry2

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF,quarter_CBSAFP
144,Industrial,2018 Q1,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,202634.593750,0.121489,16597775.0,82.98,4.0,54000.0,0.029080,0.026570,636770.0,2018 Q1&19660
145,Industrial,2018 Q2,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,203406.906250,0.123515,41943208.0,71.42,3.0,36000.0,0.032372,0.028699,688304.0,2018 Q2&19660
146,Industrial,2018 Q3,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,204901.390625,0.124135,5165600.0,44.95,4.0,37160.0,0.031292,0.030832,739480.0,2018 Q3&19660
147,Industrial,2018 Q4,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,205294.500000,0.125630,5367000.0,46.90,5.0,93120.0,0.043182,0.043182,1035788.0,2018 Q4&19660
148,Industrial,2019 Q1,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,205313.906250,0.125999,1646000.0,72.46,3.0,124960.0,0.046645,0.043040,1033872.0,2019 Q1&19660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3858,Industrial,2020 Q4,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,31842.000000,0.138026,0.0,NaN,7.0,81600.0,0.021936,0.021936,34530.0,2020 Q4&45540
3859,Industrial,2021 Q1,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,31496.189453,0.136862,1400000.0,100.57,7.0,81600.0,0.013023,0.013023,20500.0,2021 Q1&45540
3860,Industrial,2021 Q2,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,31540.269531,0.142301,5357195.0,73.60,7.0,81600.0,0.048391,0.048391,76172.0,2021 Q2&45540
3861,Industrial,2021 Q3,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,31965.048828,0.156650,3750000.0,161.18,2.0,21600.0,0.054643,0.054643,89292.0,2021 Q3&45540


# merge case and death to costar.


In [99]:
industry3=industry2.merge(pd_quarter_CBSAFP2case_death_current,left_on='quarter_CBSAFP',right_on='quarter_cbsa',how='left')

retail3=retail2.merge(pd_quarter_CBSAFP2case_death_current,left_on='quarter_CBSAFP',right_on='quarter_cbsa',how='left')

office3=office2.merge(pd_quarter_CBSAFP2case_death_current,left_on='quarter_CBSAFP',right_on='quarter_cbsa',how='left')


In [102]:
office3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384 entries, 0 to 383
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Property Class Name           384 non-null    object 
 1   Period                        384 non-null    object 
 2   Slice                         384 non-null    object 
 3   As Of                         384 non-null    object 
 4   Geography Name                384 non-null    object 
 5   CBSA Code                     384 non-null    int64  
 6   Property Type                 384 non-null    object 
 7   Forecast Scenario             384 non-null    object 
 8   Geography Type                384 non-null    object 
 9   Geography Code                384 non-null    object 
 10  Appreciation Return           384 non-null    float64
 11  Asset Value                   384 non-null    float64
 12  Availability Rate             384 non-null    float64
 13  Avail

## 取对数，用新冠数字的对数作为x变量。

In [111]:
#取对数 industry
industry3['ln_cases_current']=industry3['current_case']
industry3['ln_cases_current'][(industry3.current_case.notnull())&(industry3['ln_cases_current']!=0)]=np.log(industry3['current_case'][(industry3.current_case.notnull())&(industry3['ln_cases_current']!=0)])


industry3['ln_death_current']=industry3['current_death']
industry3['ln_death_current'][(industry3.current_death.notnull())&(industry3['ln_death_current']!=0)]=np.log(industry3['current_death'][(industry3.current_death.notnull())&(industry3['ln_death_current']!=0)])


#取对数 retail
retail3['ln_cases_current']=retail3['current_case']
retail3['ln_cases_current'][(retail3.current_case.notnull())&(retail3['ln_cases_current']!=0)]=np.log(retail3['current_case'][(retail3.current_case.notnull())&(retail3['ln_cases_current']!=0)])


retail3['ln_death_current']=retail3['current_death']
retail3['ln_death_current'][(retail3.current_death.notnull())&(retail3['ln_death_current']!=0)]=np.log(retail3['current_death'][(retail3.current_death.notnull())&(retail3['ln_death_current']!=0)])

#取对数 office
office3['ln_cases_current']=office3['current_case']
office3['ln_cases_current'][(office3.current_case.notnull())&(office3['ln_cases_current']!=0)]=np.log(office3['current_case'][(office3.current_case.notnull())&(office3['ln_cases_current']!=0)])


office3['ln_death_current']=office3['current_death']
office3['ln_death_current'][(office3.current_death.notnull())&(office3['ln_death_current']!=0)]=np.log(office3['current_death'][(office3.current_death.notnull())&(office3['ln_death_current']!=0)])






/Users/yoga/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-111-082dd0d23bd6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry3['ln_cases_current'][(industry3.current_case.notnull())&(industry3['ln_cases_current']!=0)]=np.log(industry3['current_case'][(industry3.current_case.notnull())&(industry3['ln_cases_current']!=0)])
<ipython-input-111-082dd0d23bd6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry3['ln_death_current'][(industry3.current_death.notnull())&(ind

In [123]:


office3[['ln_cases_current','ln_death_current']] = office3[['ln_cases_current','ln_death_current']].fillna(0)
retail3[['ln_cases_current','ln_death_current']] = retail3[['ln_cases_current','ln_death_current']].fillna(0)
industry3[['ln_cases_current','ln_death_current']] = industry3[['ln_cases_current','ln_death_current']].fillna(0)


#office3.to_csv('31_office_covid_2018-2021.csv',index=0)

#retail3.to_csv('32_retail_covid_2018-2021.csv',index=0)

#industry3.to_csv('33_industry_covid_2018-2021.csv',index=0)



## 进行fixed effect回归分析。

##目标处理数据，然后用stata进行fixed effect

In [181]:
office3=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_office_covid_2018-2021.csv')
retail3=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_retail_covid_2018-2021.csv')
industry3=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/33_industry_covid_2018-2021.csv')



In [ ]:
period=office3['Period'].unique().tolist()
period

In [188]:
t=list(range(1,17))
period2t=zip(period, t)
period2t
dict_period2t = dict(period2t)
dict_period2t
new_data = {'Period':list(dict_period2t.keys()), 'TimeIndex': list(dict_period2t.values())}

df = pd.DataFrame.from_dict(new_data)
df

,Period,TimeIndex
0,2018 Q1,1
1,2018 Q2,2
2,2018 Q3,3
3,2018 Q4,4
4,2019 Q1,5
5,2019 Q2,6
6,2019 Q3,7
7,2019 Q4,8
8,2020 Q1,9
9,2020 Q2,10


In [191]:
office4=office3.merge(df, left_on='Period',right_on='Period', how ='left')
retail4=retail3.merge(df, left_on='Period',right_on='Period', how ='left')
industry4=industry3.merge(df, left_on='Period',right_on='Period', how ='left')


In [194]:
#office4.to_csv('31_2_office_covid_2018-2021.csv',index=0)

#retail4.to_csv('32_2_retail_covid_2018-2021.csv',index=0)

#industry4.to_csv('33_2_industry_covid_2018-2021.csv',index=0)

# 增加facial mask and stay at home


## 导入原来的数据合并到新数据中

In [237]:
mask=pd.read_excel('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/91_old_data_used for extracting facial mask and stay at home20220507.xlsx')
office4=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_2_office_covid_2018-2021.csv')
retail4=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_2_retail_covid_2018-2021.csv')
industry4=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/33_2_industry_covid_2018-2021.csv')




In [238]:
mask.columns.tolist()

['Property Class Name',
 'Period',
 'Slice',
 'As Of',
 'Geography Name',
 'CBSA Code',
 'Property Type',
 'Forecast Scenario',
 'Geography Type',
 'Geography Code',
 'Appreciation Return',
 'Asset Value',
 'Availability Rate',
 'Available SF',
 'Average Sale Price',
 'Cap Rate',
 'Cap Rate Transactions',
 'Capital Value Index',
 'Demand SF',
 'Demolished SF',
 'Existing Buildings',
 'Gross Delivered Buildings',
 'Gross Delivered SF',
 'Households',
 'Income Return',
 'Industrial Employment',
 'Inventory SF',
 'Leasing Activity SF',
 'Market Cap Rate',
 'Market Rent Growth',
 'Market Rent Growth 12 Mo',
 'Market Rent Index',
 'Market Rent/SF',
 'Market Sale Price Growth',
 'Market Sale Price Index',
 'Market Sale Price Per SF',
 'Median Cap Rate',
 'Median Household Income',
 'Median Price/Bldg SF',
 'Net Absorption SF',
 'Net Absorption SF 12 Mo',
 'Net Absorption SF Direct',
 'Net Absorption SF Sublet',
 'Net Delivered SF',
 'Net Delivered SF 12 Mo',
 'NOI Index',
 'Occupancy Rate',


In [239]:
mask2=mask[['Period','CBSA Code','percentage_stay at home','percentage_facial mask']]

mask2['quarter_CBSAFP']=mask2['Period'].astype(str)+"&"+mask2['CBSA Code'].astype(int).astype(str)

mask2

<ipython-input-239-444b926b4fdb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mask2['quarter_CBSAFP']=mask2['Period'].astype(str)+"&"+mask2['CBSA Code'].astype(int).astype(str)


,Period,CBSA Code,percentage_stay at home,percentage_facial mask,quarter_CBSAFP
0,2019 Q1,19660,NaN,NaN,2019 Q1&19660
1,2019 Q2,19660,NaN,NaN,2019 Q2&19660
2,2019 Q3,19660,NaN,NaN,2019 Q3&19660
3,2019 Q4,19660,NaN,NaN,2019 Q4&19660
4,2020 Q1,19660,0.0,0.0,2020 Q1&19660
...,...,...,...,...,...
187,2019 Q4,45540,NaN,NaN,2019 Q4&45540
188,2020 Q1,45540,0.0,0.0,2020 Q1&45540
189,2020 Q2,45540,0.0,0.0,2020 Q2&45540
190,2020 Q3,45540,0.0,0.0,2020 Q3&45540


In [240]:
mask3=mask2[['percentage_stay at home','percentage_facial mask','quarter_CBSAFP']]
mask3

,percentage_stay at home,percentage_facial mask,quarter_CBSAFP
0,NaN,NaN,2019 Q1&19660
1,NaN,NaN,2019 Q2&19660
2,NaN,NaN,2019 Q3&19660
3,NaN,NaN,2019 Q4&19660
4,0.0,0.0,2020 Q1&19660
...,...,...,...
187,NaN,NaN,2019 Q4&45540
188,0.0,0.0,2020 Q1&45540
189,0.0,0.0,2020 Q2&45540
190,0.0,0.0,2020 Q3&45540


In [241]:
mask3.describe()

,percentage_stay at home,percentage_facial mask
count,96.000000,96.000000
mean,19.763423,26.148626
std,38.725825,40.760137
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,6.868132,47.009348
max,100.000000,100.000000


In [242]:
#merge mask into office retail and industry

office5=office4.merge(mask3, left_on='quarter_CBSAFP',right_on='quarter_CBSAFP', how ='left')
retail5=retail4.merge(mask3, left_on='quarter_CBSAFP',right_on='quarter_CBSAFP', how ='left')
industry5=industry4.merge(mask3, left_on='quarter_CBSAFP',right_on='quarter_CBSAFP', how ='left')

In [ ]:
#fill nan with 0
office5.update(office5[['percentage_stay at home','percentage_facial mask']].fillna(0))
retail5.update(retail5[['percentage_stay at home','percentage_facial mask']].fillna(0))
industry5.update(industry5[['percentage_stay at home','percentage_facial mask']].fillna(0))


# 新增季度的dummy variable


In [244]:

#add quarter dummy variables
dummies=pd.get_dummies(office5['Period']).rename(columns=lambda x: 'Period'+str(x))

office6=pd.concat([office5,dummies],axis=1)

#plan to add quarterly column
office6.loc[:,'Q1']=office6.loc[:,'Period2018 Q1'].add(office6.loc[:,'Period2019 Q1']).add(office6.loc[:,'Period2020 Q1']).add(office6.loc[:,'Period2021 Q1'])

office6.loc[:,'Q2']=office6.loc[:,'Period2018 Q2'].add(office6.loc[:,'Period2019 Q2']).add(office6.loc[:,'Period2020 Q2']).add(office6.loc[:,'Period2021 Q2'])

office6.loc[:,'Q3']=office6.loc[:,'Period2018 Q3'].add(office6.loc[:,'Period2019 Q3']).add(office6.loc[:,'Period2020 Q3']).add(office6.loc[:,'Period2021 Q3'])

office6.loc[:,'Q4']=office6.loc[:,'Period2018 Q4'].add(office6.loc[:,'Period2019 Q4']).add(office6.loc[:,'Period2020 Q4']).add(office6.loc[:,'Period2021 Q4'])


In [246]:
#same to retail
#add quarter dummy variables
dummies=pd.get_dummies(retail5['Period']).rename(columns=lambda x: 'Period'+str(x))

retail6=pd.concat([retail5,dummies],axis=1)

#plan to add quarterly column
retail6.loc[:,'Q1']=retail6.loc[:,'Period2018 Q1'].add(retail6.loc[:,'Period2019 Q1']).add(retail6.loc[:,'Period2020 Q1']).add(retail6.loc[:,'Period2021 Q1'])

retail6.loc[:,'Q2']=retail6.loc[:,'Period2018 Q2'].add(retail6.loc[:,'Period2019 Q2']).add(retail6.loc[:,'Period2020 Q2']).add(retail6.loc[:,'Period2021 Q2'])

retail6.loc[:,'Q3']=retail6.loc[:,'Period2018 Q3'].add(retail6.loc[:,'Period2019 Q3']).add(retail6.loc[:,'Period2020 Q3']).add(retail6.loc[:,'Period2021 Q3'])

retail6.loc[:,'Q4']=retail6.loc[:,'Period2018 Q4'].add(retail6.loc[:,'Period2019 Q4']).add(retail6.loc[:,'Period2020 Q4']).add(retail6.loc[:,'Period2021 Q4'])



In [249]:
#same to industry

dummies=pd.get_dummies(industry5['Period']).rename(columns=lambda x: 'Period'+str(x))

industry6=pd.concat([industry5,dummies],axis=1)

#plan to add quarterly column
industry6.loc[:,'Q1']=industry6.loc[:,'Period2018 Q1'].add(industry6.loc[:,'Period2019 Q1']).add(industry6.loc[:,'Period2020 Q1']).add(industry6.loc[:,'Period2021 Q1'])

industry6.loc[:,'Q2']=industry6.loc[:,'Period2018 Q2'].add(industry6.loc[:,'Period2019 Q2']).add(industry6.loc[:,'Period2020 Q2']).add(industry6.loc[:,'Period2021 Q2'])

industry6.loc[:,'Q3']=industry6.loc[:,'Period2018 Q3'].add(industry6.loc[:,'Period2019 Q3']).add(industry6.loc[:,'Period2020 Q3']).add(industry6.loc[:,'Period2021 Q3'])

industry6.loc[:,'Q4']=industry6.loc[:,'Period2018 Q4'].add(industry6.loc[:,'Period2019 Q4']).add(industry6.loc[:,'Period2020 Q4']).add(industry6.loc[:,'Period2021 Q4'])




In [257]:
#office6.to_csv('31_3_office_covid_2018-2021.csv',index=0)

#retail6.to_csv('32_3_retail_covid_2018-2021.csv',index=0)

#industry6.to_csv('33_3_industry_covid_2018-2021.csv',index=0)


# 新增新冠滞后一期的数据

In [3]:
office6=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_3_office_covid_2018-2021.csv')
retail6=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_3_retail_covid_2018-2021.csv')
industry6=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/33_3_industry_covid_2018-2021.csv')


In [5]:
office6.columns.tolist()

['Property Class Name',
 'Period',
 'Slice',
 'As Of',
 'Geography Name',
 'CBSA Code',
 'Property Type',
 'Forecast Scenario',
 'Geography Type',
 'Geography Code',
 'Appreciation Return',
 'Asset Value',
 'Availability Rate',
 'Available SF',
 'Average Sale Price',
 'Cap Rate',
 'Cap Rate Transactions',
 'Capital Value Index',
 'Demand SF',
 'Demolished SF',
 'Existing Buildings',
 'Gross Delivered Buildings',
 'Gross Delivered SF',
 'Households',
 'Income Return',
 'Industrial Employment',
 'Inventory SF',
 'Leasing Activity SF',
 'Market Cap Rate',
 'Market Rent Growth',
 'Market Rent Growth 12 Mo',
 'Market Rent Index',
 'Market Rent/SF',
 'Market Sale Price Growth',
 'Market Sale Price Index',
 'Market Sale Price Per SF',
 'Median Cap Rate',
 'Median Household Income',
 'Median Price/Bldg SF',
 'Net Absorption SF',
 'Net Absorption SF 12 Mo',
 'Net Absorption SF Direct',
 'Net Absorption SF Sublet',
 'Net Delivered SF',
 'Net Delivered SF 12 Mo',
 'NOI Index',
 'Occupancy Rate',


In [48]:

lncovid=office6[['Period','CBSA Code','ln_cases_current','ln_death_current']]
lncovid

,Period,CBSA Code,ln_cases_current,ln_death_current
0,2018 Q1,19660,0.000000,0.000000
1,2018 Q2,19660,0.000000,0.000000
2,2018 Q3,19660,0.000000,0.000000
3,2018 Q4,19660,0.000000,0.000000
4,2019 Q1,19660,0.000000,0.000000
...,...,...,...,...
379,2020 Q4,45540,7.927324,3.912023
380,2021 Q1,45540,8.217439,4.919981
381,2021 Q2,45540,6.586172,3.295837
382,2021 Q3,45540,8.406262,4.969813


In [49]:
#目标是产生下一期的时间序列，然后合并到cbsa code, 然后和表格进行merge，就可以了。
period=office6['Period'].unique().tolist()
period

['2018 Q1',
 '2018 Q2',
 '2018 Q3',
 '2018 Q4',
 '2019 Q1',
 '2019 Q2',
 '2019 Q3',
 '2019 Q4',
 '2020 Q1',
 '2020 Q2',
 '2020 Q3',
 '2020 Q4',
 '2021 Q1',
 '2021 Q2',
 '2021 Q3',
 '2021 Q4']

In [50]:
period[1][-1]

'2'

In [51]:
period2next={}
for i in range(len(period)):
    qi=period[i][-1]
    if qi!="4":
        period2next[period[i]]=period[i][:6]+str(int(qi)+1)
    else:
        period2next[period[i]]='20'+str(int(period[i][2:4])+1)+" Q1"
        
        


In [52]:
period2next

{'2018 Q1': '2018 Q2',
 '2018 Q2': '2018 Q3',
 '2018 Q3': '2018 Q4',
 '2018 Q4': '2019 Q1',
 '2019 Q1': '2019 Q2',
 '2019 Q2': '2019 Q3',
 '2019 Q3': '2019 Q4',
 '2019 Q4': '2020 Q1',
 '2020 Q1': '2020 Q2',
 '2020 Q2': '2020 Q3',
 '2020 Q3': '2020 Q4',
 '2020 Q4': '2021 Q1',
 '2021 Q1': '2021 Q2',
 '2021 Q2': '2021 Q3',
 '2021 Q3': '2021 Q4',
 '2021 Q4': '2022 Q1'}

In [53]:
pd_period2next = pd.DataFrame(list(period2next.items()),columns = ['current', 'next'])


In [54]:
lncovid2=lncovid.merge(pd_period2next,left_on='Period',right_on='current',how='left')
lncovid2

,Period,CBSA Code,ln_cases_current,ln_death_current,current,next
0,2018 Q1,19660,0.000000,0.000000,2018 Q1,2018 Q2
1,2018 Q2,19660,0.000000,0.000000,2018 Q2,2018 Q3
2,2018 Q3,19660,0.000000,0.000000,2018 Q3,2018 Q4
3,2018 Q4,19660,0.000000,0.000000,2018 Q4,2019 Q1
4,2019 Q1,19660,0.000000,0.000000,2019 Q1,2019 Q2
...,...,...,...,...,...,...
379,2020 Q4,45540,7.927324,3.912023,2020 Q4,2021 Q1
380,2021 Q1,45540,8.217439,4.919981,2021 Q1,2021 Q2
381,2021 Q2,45540,6.586172,3.295837,2021 Q2,2021 Q3
382,2021 Q3,45540,8.406262,4.969813,2021 Q3,2021 Q4


In [55]:
lncovid2['nest_quarter_CBSAFP']=lncovid2['next'].astype(str)+"&"+lncovid2['CBSA Code'].astype(int).astype(str)




,Period,CBSA Code,ln_cases_current,ln_death_current,current,next,nest_quarter_CBSAFP
0,2018 Q1,19660,0.000000,0.000000,2018 Q1,2018 Q2,2018 Q2&19660
1,2018 Q2,19660,0.000000,0.000000,2018 Q2,2018 Q3,2018 Q3&19660
2,2018 Q3,19660,0.000000,0.000000,2018 Q3,2018 Q4,2018 Q4&19660
3,2018 Q4,19660,0.000000,0.000000,2018 Q4,2019 Q1,2019 Q1&19660
4,2019 Q1,19660,0.000000,0.000000,2019 Q1,2019 Q2,2019 Q2&19660
...,...,...,...,...,...,...,...
379,2020 Q4,45540,7.927324,3.912023,2020 Q4,2021 Q1,2021 Q1&45540
380,2021 Q1,45540,8.217439,4.919981,2021 Q1,2021 Q2,2021 Q2&45540
381,2021 Q2,45540,6.586172,3.295837,2021 Q2,2021 Q3,2021 Q3&45540
382,2021 Q3,45540,8.406262,4.969813,2021 Q3,2021 Q4,2021 Q4&45540


In [58]:
lncovid3=lncovid2[['nest_quarter_CBSAFP','ln_cases_current','ln_death_current']]

lncovid3=lncovid3.rename(columns={"ln_cases_current": "ln_cases_prior", "ln_death_current": "ln_death_prior"})

lncovid3

,nest_quarter_CBSAFP,ln_cases_prior,ln_death_prior
0,2018 Q2&19660,0.000000,0.000000
1,2018 Q3&19660,0.000000,0.000000
2,2018 Q4&19660,0.000000,0.000000
3,2019 Q1&19660,0.000000,0.000000
4,2019 Q2&19660,0.000000,0.000000
...,...,...,...
379,2021 Q1&45540,7.927324,3.912023
380,2021 Q2&45540,8.217439,4.919981
381,2021 Q3&45540,6.586172,3.295837
382,2021 Q4&45540,8.406262,4.969813


In [63]:
#然后用next quarter cbsa fp作为key merge 到原来的数据里。
office7=office6.merge(lncovid3,left_on='quarter_CBSAFP',right_on='nest_quarter_CBSAFP',how='left')

retail7=retail6.merge(lncovid3,left_on='quarter_CBSAFP',right_on='nest_quarter_CBSAFP',how='left')

industry7=industry6.merge(lncovid3,left_on='quarter_CBSAFP',right_on='nest_quarter_CBSAFP',how='left')

In [65]:
#检查通过没有问题
#industry7 retail7 office7
retail7[['Period',
 'Geography Name',
 'CBSA Code','ln_cases_current','ln_cases_prior','ln_death_current','ln_death_prior']].tail(10)

,Period,Geography Name,CBSA Code,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior
374,2019 Q3,The Villages - FL,45540,0.000000,0.000000,0.000000,0.000000
375,2019 Q4,The Villages - FL,45540,0.000000,0.000000,0.000000,0.000000
376,2020 Q1,The Villages - FL,45540,3.931826,0.000000,0.000000,0.000000
377,2020 Q2,The Villages - FL,45540,5.683580,3.931826,2.833213,0.000000
378,2020 Q3,The Villages - FL,45540,7.584773,5.683580,3.970292,2.833213
379,2020 Q4,The Villages - FL,45540,7.927324,7.584773,3.912023,3.970292
380,2021 Q1,The Villages - FL,45540,8.217439,7.927324,4.919981,3.912023
381,2021 Q2,The Villages - FL,45540,6.586172,8.217439,3.295837,4.919981
382,2021 Q3,The Villages - FL,45540,8.406262,6.586172,4.969813,3.295837
383,2021 Q4,The Villages - FL,45540,7.254885,8.406262,3.850148,4.969813


In [120]:
industry7[['ln_cases_prior','ln_death_prior']] = industry7[['ln_cases_prior','ln_death_prior']].fillna(0)

retail7[['ln_cases_prior','ln_death_prior']] = retail7[['ln_cases_prior','ln_death_prior']].fillna(0)

office7[['ln_cases_prior','ln_death_prior']] = office7[['ln_cases_prior','ln_death_prior']].fillna(0)



In [121]:
#office7.to_csv('31_4_office_covid_2018-2021.csv',index=0)

#retail7.to_csv('32_4_retail_covid_2018-2021.csv',index=0)

#industry7.to_csv('33_4_industry_covid_2018-2021.csv',index=0)

# 进行数据拼接--useless

i.e.,2018,2019,2020的数据都用20210118更新的数据，然后2021数据用20220118更新的数据

In [67]:
old_office=pd.read_excel('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/costar_updated_20210118_in_the_first_submission_data/1  office_all_variable_2019-2020_both costar and covid.xlsx')
old_retail=pd.read_excel('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/costar_updated_20210118_in_the_first_submission_data/1_retail combine_all_variable_2019-2020.xlsx')
old_industry=pd.read_excel('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/costar_updated_20210118_in_the_first_submission_data/1_industry_all variable_from costar2019-2020.xlsx')




In [106]:
office20210115=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/84 new costar data 01152021/dataupdate/Office_01-15-2021.xlsx')
retail20210115=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/84 new costar data 01152021/dataupdate/Retail_01-15-2021.xlsx')
industry20210115=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/84 new costar data 01152021/dataupdate/Industrial_01-15-2021.xlsx')





In [73]:
old_office[['Period','Market Rent Growth','Vacancy Rate','Market Sale Price Growth','Sales Volume Transactions']]

,Period,Market Rent Growth,Vacancy Rate,Market Sale Price Growth,Sales Volume Transactions
0,2019 Q1,-0.004333,0.047025,0.022646,27.0
1,2019 Q2,0.000678,0.045713,0.012133,25.0
2,2019 Q3,0.001370,0.045060,0.008434,31.0
3,2019 Q4,0.011094,0.038580,0.011606,31.0
4,2020 Q1,0.004794,0.040858,0.019041,26.0
...,...,...,...,...,...
187,2019 Q4,0.005256,0.044529,0.014346,2.0
188,2020 Q1,0.000251,0.041562,0.015486,4.0
189,2020 Q2,-0.000917,0.042182,0.016129,2.0
190,2020 Q3,-0.000794,0.033718,0.015310,2.0


In [77]:
office20210115_1920=office20210115[(office20210115['Period'].str.contains('2019 Q'))|(office20210115['Period'].str.contains('2020 Q'))]

office20210115_1920[['Market Rent Growth','Vacancy Rate','Market Sale Price Growth','Sales Volume Transactions']][:192]


,Market Rent Growth,Vacancy Rate,Market Sale Price Growth,Sales Volume Transactions
148,-0.004333,0.047025,0.022646,27.0
149,0.000678,0.045713,0.012133,25.0
150,0.001370,0.045060,0.008434,31.0
151,0.011094,0.038580,0.011606,31.0
152,0.004794,0.040858,0.019041,26.0
...,...,...,...,...
3762,0.005256,0.044529,0.014346,2.0
3763,0.000251,0.041562,0.015486,4.0
3764,-0.000917,0.042182,0.016129,2.0
3765,-0.000794,0.033718,0.015310,2.0


经过比较，第一次paper提交，采用的数据是20210115更新的costar的数据。从20210115的数据中提取2018，2019，2020的数据。


## 从20210115的costar数据中提取2018，2019，2020的数据。

In [80]:
#先尝试Office

office20210115_181920=office20210115[(office20210115['Period'].str.contains('2018 Q'))|(office20210115['Period'].str.contains('2019 Q'))|(office20210115['Period'].str.contains('2020 Q'))]
office20210115_181920_fl=office20210115_181920[office20210115_181920['Geography Name'].str.contains(' - FL')]


In [84]:
office20210115_181920_fl

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Sold Building SF,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF
144,Office,2018 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,162156.0,202598.906250,0.094606,16528521.0,101.93,8.0,194674.0,0.048415,0.046416,631316.0
145,Office,2018 Q2,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,167432.0,203477.703125,0.096747,12328509.0,73.63,9.0,195834.0,0.041848,0.040883,556056.0
146,Office,2018 Q3,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,66882.0,204844.890625,0.095005,8098079.0,121.08,7.0,32460.0,0.041179,0.040337,555604.0
147,Office,2018 Q4,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,113531.0,205306.500000,0.085824,12595560.0,110.94,4.0,16660.0,0.046052,0.045879,632659.0
148,Office,2019 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,80084.0,205315.203125,0.073354,9263195.0,115.67,2.0,12966.0,0.047025,0.045204,623966.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3762,Office,2019 Q4,All,2021 Q1,The Villages - FL,45540.0,OFF,Base Case,Metro,G39580,...,1650.0,32390.701172,0.061827,211523.0,128.20,2.0,35000.0,0.044529,0.044529,77493.0
3763,Office,2020 Q1,All,2021 Q1,The Villages - FL,45540.0,OFF,Base Case,Metro,G39580,...,5963.0,32458.339844,0.063073,1416850.0,237.61,2.0,13500.0,0.041562,0.040559,71598.0
3764,Office,2020 Q2,All,2021 Q1,The Villages - FL,45540.0,OFF,Base Case,Metro,G39580,...,13000.0,31103.900391,0.063744,1220000.0,93.85,2.0,13500.0,0.042182,0.041512,73280.0
3765,Office,2020 Q3,All,2021 Q1,The Villages - FL,45540.0,OFF,Base Case,Metro,G39580,...,4352.0,31948.240234,0.062837,472500.0,108.57,2.0,13500.0,0.033718,0.033094,58420.0


## 从costar 2022数据中提取2021年一年的数据。

In [87]:
#从costar原始数据中提取2021年的一整年的数据，然后把新的数据里面自己生成的数据，merge到这个only costar的数据里。
office20220118=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/73_costar data 20220118/reupdatecostardatatoincludethewholeyearof2021/Office_01-18-2022.xlsx')
retail20220118=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/73_costar data 20220118/reupdatecostardatatoincludethewholeyearof2021/Retail_01-18-2022.xlsx')
industry20220118=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/73_costar data 20220118/reupdatecostardatatoincludethewholeyearof2021/Industrial_01-18-2022.xlsx')





### 拼接Office数据

In [88]:
office20220118_2021=office20220118[office20220118['Period'].str.contains('2021 Q')]

office20220118_2021_fl=office20220118_2021[office20220118_2021['Geography Name'].str.contains(' - FL')]


In [89]:
office20220118_2021_fl

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Sold Building SF,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF
156,Office,2021 Q1,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,204668.0,2.030984e+05,0.075934,22471944.0,118.16,2.0,51088.0,0.049177,0.047786,681659.0
157,Office,2021 Q2,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,292384.0,2.035995e+05,0.080540,33328060.0,120.40,3.0,61981.0,0.047336,0.046372,661485.0
158,Office,2021 Q3,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,288557.0,2.058527e+05,0.084914,33647060.0,130.56,2.0,50833.0,0.050615,0.049480,706370.0
159,Office,2021 Q4,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,126214.0,2.070538e+05,0.094394,14888180.0,162.53,2.0,50833.0,0.052019,0.051261,731801.0
317,Office,2021 Q1,All,2022 Q1,Fort Lauderdale - FL,22744,Office,Base Case,Metro,G41715,...,1049713.0,8.181205e+05,0.049045,189948016.0,201.14,6.0,595360.0,0.114005,0.107927,7800056.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,Office,2021 Q4,All,2022 Q1,Tampa - FL,45300,Office,Base Case,Metro,G39577,...,3171190.0,1.415920e+06,0.095912,600994496.0,206.07,22.0,604695.0,0.087146,0.078684,10015801.0
3859,Office,2021 Q1,All,2022 Q1,The Villages - FL,45540,Office,Base Case,Metro,G39580,...,17923.0,3.149619e+04,0.066805,1510000.0,102.81,1.0,45000.0,0.036521,0.036418,64241.0
3860,Office,2021 Q2,All,2022 Q1,The Villages - FL,45540,Office,Base Case,Metro,G39580,...,3424.0,3.154027e+04,0.068864,839480.0,245.18,1.0,45000.0,0.044187,0.042950,75763.0
3861,Office,2021 Q3,All,2022 Q1,The Villages - FL,45540,Office,Base Case,Metro,G39580,...,10838.0,3.196505e+04,0.079519,149847.0,13.83,0.0,NaN,0.046213,0.046212,83597.0


In [90]:
combine_office1821=pd.concat([office20210115_181920_fl, office20220118_2021_fl])

In [93]:
combine_office1821

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Sold Building SF,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF
144,Office,2018 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,162156.0,2.025989e+05,0.094606,16528521.0,101.93,8.0,194674.0,0.048415,0.046416,631316.0
145,Office,2018 Q2,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,167432.0,2.034777e+05,0.096747,12328509.0,73.63,9.0,195834.0,0.041848,0.040883,556056.0
146,Office,2018 Q3,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,66882.0,2.048449e+05,0.095005,8098079.0,121.08,7.0,32460.0,0.041179,0.040337,555604.0
147,Office,2018 Q4,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,113531.0,2.053065e+05,0.085824,12595560.0,110.94,4.0,16660.0,0.046052,0.045879,632659.0
148,Office,2019 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,80084.0,2.053152e+05,0.073354,9263195.0,115.67,2.0,12966.0,0.047025,0.045204,623966.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,Office,2021 Q4,All,2022 Q1,Tampa - FL,45300.0,Office,Base Case,Metro,G39577,...,3171190.0,1.415920e+06,0.095912,600994496.0,206.07,22.0,604695.0,0.087146,0.078684,10015801.0
3859,Office,2021 Q1,All,2022 Q1,The Villages - FL,45540.0,Office,Base Case,Metro,G39580,...,17923.0,3.149619e+04,0.066805,1510000.0,102.81,1.0,45000.0,0.036521,0.036418,64241.0
3860,Office,2021 Q2,All,2022 Q1,The Villages - FL,45540.0,Office,Base Case,Metro,G39580,...,3424.0,3.154027e+04,0.068864,839480.0,245.18,1.0,45000.0,0.044187,0.042950,75763.0
3861,Office,2021 Q3,All,2022 Q1,The Villages - FL,45540.0,Office,Base Case,Metro,G39580,...,10838.0,3.196505e+04,0.079519,149847.0,13.83,0.0,NaN,0.046213,0.046212,83597.0


In [94]:
#从最新处理的数据中，分离出新产生的列，然后合并进来

In [99]:
#office7.columns.tolist()

In [97]:
newly_generated=office7[['quarter_CBSAFP',
 'quarter_cbsa',
 'current_case',
 'current_death',
 'ln_cases_current',
 'ln_death_current',
 'TimeIndex',
 'percentage_stay at home',
 'percentage_facial mask',
 'Period2018 Q1',
 'Period2018 Q2',
 'Period2018 Q3',
 'Period2018 Q4',
 'Period2019 Q1',
 'Period2019 Q2',
 'Period2019 Q3',
 'Period2019 Q4',
 'Period2020 Q1',
 'Period2020 Q2',
 'Period2020 Q3',
 'Period2020 Q4',
 'Period2021 Q1',
 'Period2021 Q2',
 'Period2021 Q3',
 'Period2021 Q4',
 'Q1',
 'Q2',
 'Q3',
 'Q4',
 'nest_quarter_CBSAFP',
 'ln_cases_prior',
 'ln_death_prior']]

In [98]:
newly_generated

,quarter_CBSAFP,quarter_cbsa,current_case,current_death,ln_cases_current,ln_death_current,TimeIndex,percentage_stay at home,percentage_facial mask,Period2018 Q1,...,Period2021 Q2,Period2021 Q3,Period2021 Q4,Q1,Q2,Q3,Q4,nest_quarter_CBSAFP,ln_cases_prior,ln_death_prior
0,2018 Q1&19660,NaN,NaN,NaN,0.000000,0.000000,1,0.0,0.0,1,...,0,0,0,1,0,0,0,NaN,NaN,NaN
1,2018 Q2&19660,NaN,NaN,NaN,0.000000,0.000000,2,0.0,0.0,0,...,0,0,0,0,1,0,0,2018 Q2&19660,0.000000,0.000000
2,2018 Q3&19660,NaN,NaN,NaN,0.000000,0.000000,3,0.0,0.0,0,...,0,0,0,0,0,1,0,2018 Q3&19660,0.000000,0.000000
3,2018 Q4&19660,NaN,NaN,NaN,0.000000,0.000000,4,0.0,0.0,0,...,0,0,0,0,0,0,1,2018 Q4&19660,0.000000,0.000000
4,2019 Q1&19660,NaN,NaN,NaN,0.000000,0.000000,5,0.0,0.0,0,...,0,0,0,1,0,0,0,2019 Q1&19660,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,2020 Q4&45540,2020 Q4&45540,2772.0,50.0,7.927324,3.912023,12,0.0,0.0,0,...,0,0,0,0,0,0,1,2020 Q4&45540,7.584773,3.970292
380,2021 Q1&45540,2021 Q1&45540,3705.0,137.0,8.217439,4.919981,13,0.0,0.0,0,...,0,0,0,1,0,0,0,2021 Q1&45540,7.927324,3.912023
381,2021 Q2&45540,2021 Q2&45540,725.0,27.0,6.586172,3.295837,14,0.0,0.0,0,...,1,0,0,0,1,0,0,2021 Q2&45540,8.217439,4.919981
382,2021 Q3&45540,2021 Q3&45540,4475.0,144.0,8.406262,4.969813,15,0.0,0.0,0,...,0,1,0,0,0,1,0,2021 Q3&45540,6.586172,3.295837


In [101]:
combine_office1821['quarter_CBSAFP']=combine_office1821['Period'].astype(str)+"&"+combine_office1821['CBSA Code'].astype(int).astype(str)


,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF,quarter_CBSAFP
144,Office,2018 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,2.025989e+05,0.094606,16528521.0,101.93,8.0,194674.0,0.048415,0.046416,631316.0,2018 Q1&19660
145,Office,2018 Q2,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,2.034777e+05,0.096747,12328509.0,73.63,9.0,195834.0,0.041848,0.040883,556056.0,2018 Q2&19660
146,Office,2018 Q3,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,2.048449e+05,0.095005,8098079.0,121.08,7.0,32460.0,0.041179,0.040337,555604.0,2018 Q3&19660
147,Office,2018 Q4,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,2.053065e+05,0.085824,12595560.0,110.94,4.0,16660.0,0.046052,0.045879,632659.0,2018 Q4&19660
148,Office,2019 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,OFF,Base Case,Metro,G41649,...,2.053152e+05,0.073354,9263195.0,115.67,2.0,12966.0,0.047025,0.045204,623966.0,2019 Q1&19660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,Office,2021 Q4,All,2022 Q1,Tampa - FL,45300.0,Office,Base Case,Metro,G39577,...,1.415920e+06,0.095912,600994496.0,206.07,22.0,604695.0,0.087146,0.078684,10015801.0,2021 Q4&45300
3859,Office,2021 Q1,All,2022 Q1,The Villages - FL,45540.0,Office,Base Case,Metro,G39580,...,3.149619e+04,0.066805,1510000.0,102.81,1.0,45000.0,0.036521,0.036418,64241.0,2021 Q1&45540
3860,Office,2021 Q2,All,2022 Q1,The Villages - FL,45540.0,Office,Base Case,Metro,G39580,...,3.154027e+04,0.068864,839480.0,245.18,1.0,45000.0,0.044187,0.042950,75763.0,2021 Q2&45540
3861,Office,2021 Q3,All,2022 Q1,The Villages - FL,45540.0,Office,Base Case,Metro,G39580,...,3.196505e+04,0.079519,149847.0,13.83,0.0,NaN,0.046213,0.046212,83597.0,2021 Q3&45540


In [102]:
#generate new data

pingjie_office=combine_office1821.merge(newly_generated,left_on='quarter_CBSAFP',right_on='quarter_CBSAFP',how='left')

### 拼接retail 数据

In [107]:
retail20210115_181920=retail20210115[(retail20210115['Period'].str.contains('2018 Q'))|(retail20210115['Period'].str.contains('2019 Q'))|(retail20210115['Period'].str.contains('2020 Q'))]
retail20210115_181920_fl=retail20210115_181920[retail20210115_181920['Geography Name'].str.contains(' - FL')]

retail20220118_2021=retail20220118[retail20220118['Period'].str.contains('2021 Q')]

retail20220118_2021_fl=retail20220118_2021[retail20220118_2021['Geography Name'].str.contains(' - FL')]

combine_retail1821=pd.concat([retail20210115_181920_fl, retail20220118_2021_fl])

combine_retail1821['quarter_CBSAFP']=combine_retail1821['Period'].astype(str)+"&"+combine_retail1821['CBSA Code'].astype(int).astype(str)


pingjie_retail=combine_retail1821.merge(newly_generated,left_on='quarter_CBSAFP',right_on='quarter_CBSAFP',how='left')


In [108]:
pingjie_retail

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Period2021 Q2,Period2021 Q3,Period2021 Q4,Q1,Q2,Q3,Q4,nest_quarter_CBSAFP,ln_cases_prior,ln_death_prior
0,Retail,2018 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,RET,Base Case,Metro,G41649,...,0,0,0,1,0,0,0,NaN,NaN,NaN
1,Retail,2018 Q2,All,2021 Q1,Daytona Beach - FL,19660.0,RET,Base Case,Metro,G41649,...,0,0,0,0,1,0,0,2018 Q2&19660,0.000000,0.000000
2,Retail,2018 Q3,All,2021 Q1,Daytona Beach - FL,19660.0,RET,Base Case,Metro,G41649,...,0,0,0,0,0,1,0,2018 Q3&19660,0.000000,0.000000
3,Retail,2018 Q4,All,2021 Q1,Daytona Beach - FL,19660.0,RET,Base Case,Metro,G41649,...,0,0,0,0,0,0,1,2018 Q4&19660,0.000000,0.000000
4,Retail,2019 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,RET,Base Case,Metro,G41649,...,0,0,0,1,0,0,0,2019 Q1&19660,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Retail,2021 Q4,All,2022 Q1,Tampa - FL,45300.0,Retail,Base Case,Metro,G39577,...,0,0,1,0,0,0,1,2021 Q4&45300,12.118852,7.796058
380,Retail,2021 Q1,All,2022 Q1,The Villages - FL,45540.0,Retail,Base Case,Metro,G39580,...,0,0,0,1,0,0,0,2021 Q1&45540,7.927324,3.912023
381,Retail,2021 Q2,All,2022 Q1,The Villages - FL,45540.0,Retail,Base Case,Metro,G39580,...,1,0,0,0,1,0,0,2021 Q2&45540,8.217439,4.919981
382,Retail,2021 Q3,All,2022 Q1,The Villages - FL,45540.0,Retail,Base Case,Metro,G39580,...,0,1,0,0,0,1,0,2021 Q3&45540,6.586172,3.295837


### 拼接industry 数据

In [112]:
industry20210115_181920=industry20210115[(industry20210115['Period'].str.contains('2018 Q'))|(industry20210115['Period'].str.contains('2019 Q'))|(industry20210115['Period'].str.contains('2020 Q'))]
industry20210115_181920_fl=industry20210115_181920[industry20210115_181920['Geography Name'].str.contains(' - FL')]

industry20220118_2021=industry20220118[industry20220118['Period'].str.contains('2021 Q')]

industry20220118_2021_fl=industry20220118_2021[industry20220118_2021['Geography Name'].str.contains(' - FL')]

combine_industry1821=pd.concat([industry20210115_181920_fl, industry20220118_2021_fl])

combine_industry1821['quarter_CBSAFP']=combine_industry1821['Period'].astype(str)+"&"+combine_industry1821['CBSA Code'].astype(int).astype(str)


pingjie_industry=combine_industry1821.merge(newly_generated,left_on='quarter_CBSAFP',right_on='quarter_CBSAFP',how='left')


In [113]:
pingjie_industry

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Period2021 Q2,Period2021 Q3,Period2021 Q4,Q1,Q2,Q3,Q4,nest_quarter_CBSAFP,ln_cases_prior,ln_death_prior
0,Industrial,2018 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,IND,Base Case,Metro,G41649,...,0,0,0,1,0,0,0,NaN,NaN,NaN
1,Industrial,2018 Q2,All,2021 Q1,Daytona Beach - FL,19660.0,IND,Base Case,Metro,G41649,...,0,0,0,0,1,0,0,2018 Q2&19660,0.000000,0.000000
2,Industrial,2018 Q3,All,2021 Q1,Daytona Beach - FL,19660.0,IND,Base Case,Metro,G41649,...,0,0,0,0,0,1,0,2018 Q3&19660,0.000000,0.000000
3,Industrial,2018 Q4,All,2021 Q1,Daytona Beach - FL,19660.0,IND,Base Case,Metro,G41649,...,0,0,0,0,0,0,1,2018 Q4&19660,0.000000,0.000000
4,Industrial,2019 Q1,All,2021 Q1,Daytona Beach - FL,19660.0,IND,Base Case,Metro,G41649,...,0,0,0,1,0,0,0,2019 Q1&19660,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Industrial,2021 Q4,All,2022 Q1,Tampa - FL,45300.0,Industrial,Base Case,Metro,G39577,...,0,0,1,0,0,0,1,2021 Q4&45300,12.118852,7.796058
380,Industrial,2021 Q1,All,2022 Q1,The Villages - FL,45540.0,Industrial,Base Case,Metro,G39580,...,0,0,0,1,0,0,0,2021 Q1&45540,7.927324,3.912023
381,Industrial,2021 Q2,All,2022 Q1,The Villages - FL,45540.0,Industrial,Base Case,Metro,G39580,...,1,0,0,0,1,0,0,2021 Q2&45540,8.217439,4.919981
382,Industrial,2021 Q3,All,2022 Q1,The Villages - FL,45540.0,Industrial,Base Case,Metro,G39580,...,0,1,0,0,0,1,0,2021 Q3&45540,6.586172,3.295837


In [119]:
#replace nan with 0 in covid prior column

pingjie_industry[['ln_cases_prior','ln_death_prior']] = pingjie_industry[['ln_cases_prior','ln_death_prior']].fillna(0)

pingjie_retail[['ln_cases_prior','ln_death_prior']] = pingjie_retail[['ln_cases_prior','ln_death_prior']].fillna(0)

pingjie_office[['ln_cases_prior','ln_death_prior']] = pingjie_office[['ln_cases_prior','ln_death_prior']].fillna(0)



In [123]:
#pingjie_office.to_csv('31_5_pingjie_office_covid_2018-2021.csv',index=0)

#pingjie_retail.to_csv('32_5_pingjie_retail_covid_2018-2021.csv',index=0)

#pingjie_industry.to_csv('33_5_pingjie_industry_covid_2018-2021.csv',index=0)

# 计算correlation相关性


In [2]:
retail=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_4_retail_covid_2018-2021.csv')

In [5]:
retail.columns

Index(['Property Class Name', 'Period', 'Slice', 'As Of', 'Geography Name',
       'CBSA Code', 'Property Type', 'Forecast Scenario', 'Geography Type',
       'Geography Code', 'All Service Type Rent Direct',
       'All Service Type Rent Overall', 'All Service Type Rent Sublet',
       'Appreciation Return', 'Asset Value', 'Availability Rate',
       'Available SF', 'Average Sale Price', 'Cap Rate',
       'Cap Rate Transactions', 'Capital Value Index', 'Demand SF',
       'Demolished SF', 'Existing Buildings', 'Existing Centers',
       'Gross Delivered Buildings', 'Gross Delivered SF', 'Households',
       'Income Return', 'Industrial Employment', 'Inventory SF',
       'Leasing Activity SF', 'Market Cap Rate', 'Market Rent Growth',
       'Market Rent Growth 12 Mo', 'Market Rent Index', 'Market Rent/SF',
       'Market Sale Price Growth', 'Market Sale Price Index',
       'Market Sale Price Per SF', 'Median Cap Rate',
       'Median Household Income', 'Median Price/Bldg SF', 'Net A

In [10]:
retail_cor=retail[['ln_cases_current', 'ln_cases_prior', 'ln_death_current','ln_death_prior',
                   'Total Employment','percentage_stay at home', 'percentage_facial mask']]

In [11]:
retail_cor

,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior,Total Employment,percentage_stay at home,percentage_facial mask
0,0.000000,0.000000,0.000000,0.000000,202634.593750,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,203406.906250,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,204901.390625,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,205294.500000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,205313.906250,0.0,0.0
...,...,...,...,...,...,...,...
379,7.927324,7.584773,3.912023,3.970292,31842.000000,0.0,0.0
380,8.217439,7.927324,4.919981,3.912023,31496.189453,0.0,0.0
381,6.586172,8.217439,3.295837,4.919981,31540.269531,0.0,0.0
382,8.406262,6.586172,4.969813,3.295837,31965.048828,0.0,0.0


In [12]:
corrMatrix = retail_cor.corr()

In [13]:
corrMatrix

,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior,Total Employment,percentage_stay at home,percentage_facial mask
ln_cases_current,1.000000,0.929994,0.973075,0.876552,0.106457,0.279194,0.343981
ln_cases_prior,0.929994,1.000000,0.956173,0.972408,0.092816,0.216451,0.315155
ln_death_current,0.973075,0.956173,1.000000,0.913185,0.137557,0.267302,0.347986
ln_death_prior,0.876552,0.972408,0.913185,1.000000,0.129157,0.159154,0.280199
Total Employment,0.106457,0.092816,0.137557,0.129157,1.000000,0.182648,0.161612
percentage_stay at home,0.279194,0.216451,0.267302,0.159154,0.182648,1.000000,0.651636
percentage_facial mask,0.343981,0.315155,0.347986,0.280199,0.161612,0.651636,1.000000


In [14]:
#corrMatrix.to_csv('399_correlation_appendix_in_paper.csv')

In [23]:
retail_cor['Million_emp']=retail_cor['Total Employment']/1000000
describe=retail_cor.describe()

<ipython-input-23-a8b10cdac90f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retail_cor['Million_emp']=retail_cor['Total Employment']/1000000


In [24]:
#describe.to_csv('398_correlation_describe.csv')

In [20]:
retail_cor.corr()

,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior,Total Employment,percentage_stay at home,percentage_facial mask,Million_emp
ln_cases_current,1.000000,0.929994,0.973075,0.876552,0.106457,0.279194,0.343981,0.106457
ln_cases_prior,0.929994,1.000000,0.956173,0.972408,0.092816,0.216451,0.315155,0.092816
ln_death_current,0.973075,0.956173,1.000000,0.913185,0.137557,0.267302,0.347986,0.137557
ln_death_prior,0.876552,0.972408,0.913185,1.000000,0.129157,0.159154,0.280199,0.129157
Total Employment,0.106457,0.092816,0.137557,0.129157,1.000000,0.182648,0.161612,1.000000
percentage_stay at home,0.279194,0.216451,0.267302,0.159154,0.182648,1.000000,0.651636,0.182648
percentage_facial mask,0.343981,0.315155,0.347986,0.280199,0.161612,0.651636,1.000000,0.161612
Million_emp,0.106457,0.092816,0.137557,0.129157,1.000000,0.182648,0.161612,1.000000


,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior,Total Employment,percentage_stay at home,percentage_facial mask,Million_emp
count,384.000000,384.000000,384.000000,384.000000,3.840000e+02,384.000000,384.000000,384.000000
mean,4.286741,3.712482,2.335957,1.996656,3.571100e+05,4.940856,6.537156,0.357110
std,4.560674,4.471949,2.694712,2.615021,4.048860e+05,21.104811,23.251490,0.404886
min,0.000000,0.000000,0.000000,0.000000,2.402740e+04,0.000000,0.000000,0.024027
25%,0.000000,0.000000,0.000000,0.000000,9.797862e+04,0.000000,0.000000,0.097979
50%,0.000000,0.000000,0.000000,0.000000,1.844490e+05,0.000000,0.000000,0.184449
75%,9.054283,8.657080,5.051362,4.738382,3.835281e+05,0.000000,0.000000,0.383528
max,12.159678,12.118852,7.796058,7.796058,1.415920e+06,100.000000,100.000000,1.415920


# 计算paper里面出现的数字

## It has caused 824k deaths as of December 2021

In [2]:
covid=pd.read_csv('/Users/yoga/Documents/Projects/1_covid19_cre/72_update regression analysis 20220122/1_us-counties_covid20220122.csv')


In [4]:
covid2021=covid[covid['date']=='2021-12-31']

In [23]:
covid2021['deaths'].sum()

824336.0

## Florida is a state with the third largest accumulated number of COVID cases ( 4.17 million) in the U.S. in 2021

In [24]:
covid2021fl=covid2021.loc[covid2021['state']=='Florida']

In [26]:
covid2021fl['cases'].sum()

4166392

## The asset value of the commercial real estate industry reached 4 billion in the fourth quarter of 2021

In [25]:
office=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/73_costar data 20220118/reupdatecostardatatoincludethewholeyearof2021/Office_01-18-2022.xlsx')
industry=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/73_costar data 20220118/reupdatecostardatatoincludethewholeyearof2021/Industrial_01-18-2022.xlsx')
retail=pd.read_excel('/Users/yoga/Documents/Projects/1_covid19_cre/73_costar data 20220118/reupdatecostardatatoincludethewholeyearof2021/Retail_01-18-2022.xlsx')

In [20]:
retail1=retail[['Geography Name','Period','Asset Value']]
retail2=retail1[retail1['Period']=='2021 Q4']

retail3=retail2['Asset Value'].sum()
retail3

280633485696.0

In [21]:
office1=office[['Geography Name','Period','Asset Value']]
office2=office1[office1['Period']=='2021 Q4']

office3=office2['Asset Value'].sum()
office3


162607961792.0

In [22]:
industry1=industry[['Geography Name','Period','Asset Value']]
industry2=industry1[industry1['Period']=='2021 Q4']

industry3=industry2['Asset Value'].sum()
industry3


180730644904.0

In [23]:
total=industry3+office3+retail3

In [25]:
total/1000000000

623.972092392

## Every one percent increase in COVID cases causes the retail vacancy rate to grow by 0.3 percent each quarter – about 352 thousand square feet.

In [6]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3864 entries, 0 to 3863
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Property Class Name            3864 non-null   object 
 1   Period                         3864 non-null   object 
 2   Slice                          3864 non-null   object 
 3   As Of                          3864 non-null   object 
 4   Geography Name                 3864 non-null   object 
 5   CBSA Code                      3864 non-null   int64  
 6   Property Type                  3864 non-null   object 
 7   Forecast Scenario              3864 non-null   object 
 8   Geography Type                 3864 non-null   object 
 9   Geography Code                 3864 non-null   object 
 10  All Service Type Rent Direct   1487 non-null   float64
 11  All Service Type Rent Overall  1487 non-null   float64
 12  All Service Type Rent Sublet   1176 non-null   f

In [26]:
#get the 2020 Q1 data
retail2=retail[retail['Period'].str.contains('2020 Q1')]
retail3=retail[retail['Period'].str.contains('2020 Q2')]
retail2

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Sold Building SF,Total Employment,Total Return,Total Sales Volume,Transaction Sale Price/SF,Under Construction Buildings,Under Construction SF,Vacancy Rate,Vacant Available %,Vacant Available SF
152,Retail,2020 Q1,All,2022 Q1,Daytona Beach - FL,19660,Retail,Base Case,Metro,G41649,...,567655.0,2.076436e+05,0.092421,86575872.0,161.15,7.0,148897.0,0.043260,0.042572,1691699.0
313,Retail,2020 Q1,All,2022 Q1,Fort Lauderdale - FL,22744,Retail,Base Case,Metro,G41715,...,721566.0,8.713660e+05,0.069432,162616624.0,278.63,45.0,913417.0,0.042585,0.039103,4275266.0
474,Retail,2020 Q1,All,2022 Q1,Fort Myers - FL,15980,Retail,Base Case,Metro,G41609,...,857931.0,2.806959e+05,0.072605,91193224.0,111.94,25.0,298268.0,0.055938,0.052618,2532393.0
635,Retail,2020 Q1,All,2022 Q1,Ft Walton Beach - FL,18880,Retail,Base Case,Metro,G41639,...,247256.0,1.196070e+05,0.069466,8625300.0,126.14,9.0,178006.0,0.025671,0.024909,489583.0
796,Retail,2020 Q1,All,2022 Q1,Gainesville - FL,23540,Retail,Base Case,Metro,G41743,...,190624.0,1.486744e+05,0.057596,12316000.0,104.60,7.0,65443.0,0.019187,0.018826,365530.0
957,Retail,2020 Q1,All,2022 Q1,Homosassa Springs - FL,26140,Retail,Base Case,Metro,G41804,...,77264.0,3.320562e+04,0.075378,6597843.0,109.20,2.0,6665.0,0.065337,0.051881,514747.0
1118,Retail,2020 Q1,All,2022 Q1,Jacksonville - FL,27260,Retail,Base Case,Metro,G41817,...,787557.0,7.319252e+05,0.090172,126094592.0,165.75,66.0,1247215.0,0.045949,0.042167,4068671.0
1279,Retail,2020 Q1,All,2022 Q1,Lakeland - FL,29460,Retail,Base Case,Metro,G39424,...,410747.0,2.413164e+05,0.077058,48159796.0,131.83,21.0,207633.0,0.035479,0.029721,933814.0
1440,Retail,2020 Q1,All,2022 Q1,Melbourne - FL,37340,Retail,Base Case,Metro,G39499,...,255452.0,2.339077e+05,0.084913,41828464.0,193.30,10.0,164677.0,0.054184,0.052177,1810976.0
1601,Retail,2020 Q1,All,2022 Q1,Miami - FL,33124,Retail,Base Case,Metro,G39457,...,774827.0,1.227054e+06,0.061674,175010416.0,261.18,47.0,2184584.0,0.039201,0.035548,4911815.0


In [27]:
retail21=retail2[['Period','Geography Name','Inventory SF']]
retail21

,Period,Geography Name,Inventory SF
152,2020 Q1,Daytona Beach - FL,39737253
313,2020 Q1,Fort Lauderdale - FL,109331087
474,2020 Q1,Fort Myers - FL,48127459
635,2020 Q1,Ft Walton Beach - FL,19654258
796,2020 Q1,Gainesville - FL,19415898
957,2020 Q1,Homosassa Springs - FL,9921534
1118,2020 Q1,Jacksonville - FL,96488785
1279,2020 Q1,Lakeland - FL,31419307
1440,2020 Q1,Melbourne - FL,34707799
1601,2020 Q1,Miami - FL,138173131


In [21]:
#用2020 q1的数据作为baseline 进行计算
retail21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 152 to 3855
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Period          24 non-null     object
 1   Geography Name  24 non-null     object
 2   Inventory SF    24 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 768.0+ bytes


In [28]:
retail21['Inventory SF'].sum()

1171986208

In [23]:
1171986208*0.0003

351595.8624

In [29]:
1171986208*0.0005

585993.104

# 根据期刊sustainability的反馈，需要增加一个statistic table--2022.09.12

In [1]:
import pandas as pd
import numpy as np

In [2]:
office=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_4_office_covid_2018-2021.csv')

In [4]:
office.columns.tolist()

['Property Class Name',
 'Period',
 'Slice',
 'As Of',
 'Geography Name',
 'CBSA Code',
 'Property Type',
 'Forecast Scenario',
 'Geography Type',
 'Geography Code',
 'Appreciation Return',
 'Asset Value',
 'Availability Rate',
 'Available SF',
 'Average Sale Price',
 'Cap Rate',
 'Cap Rate Transactions',
 'Capital Value Index',
 'Demand SF',
 'Demolished SF',
 'Existing Buildings',
 'Gross Delivered Buildings',
 'Gross Delivered SF',
 'Households',
 'Income Return',
 'Industrial Employment',
 'Inventory SF',
 'Leasing Activity SF',
 'Market Cap Rate',
 'Market Rent Growth',
 'Market Rent Growth 12 Mo',
 'Market Rent Index',
 'Market Rent/SF',
 'Market Sale Price Growth',
 'Market Sale Price Index',
 'Market Sale Price Per SF',
 'Median Cap Rate',
 'Median Household Income',
 'Median Price/Bldg SF',
 'Net Absorption SF',
 'Net Absorption SF 12 Mo',
 'Net Absorption SF Direct',
 'Net Absorption SF Sublet',
 'Net Delivered SF',
 'Net Delivered SF 12 Mo',
 'NOI Index',
 'Occupancy Rate',


In [5]:
office_st=office[['Market Rent Growth','Vacancy Rate','Sales Volume Transactions','Market Sale Price Growth',
                'ln_cases_current','ln_death_current', 'Total Employment','percentage_stay at home',
 'percentage_facial mask'
                 
                 
                 ]]

In [12]:
describe_office=office_st.describe().transpose()

In [13]:
#describe_office.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/103 subimt 3 times/first review/office_description.csv')

In [15]:
retail=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_4_retail_covid_2018-2021.csv')

In [16]:
retail_st=retail[['Market Rent Growth','Vacancy Rate','Sales Volume Transactions','Market Sale Price Growth']]

In [17]:
describe_retail=retail_st.describe().transpose()

In [18]:
#describe_retail.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/103 subimt 3 times/first review/retail_description.csv')

In [19]:
industry=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/33_4_industry_covid_2018-2021.csv')

In [20]:
industry2=industry[['Market Rent Growth','Vacancy Rate','Sales Volume Transactions','Market Sale Price Growth']]

In [21]:
industry3=industry2.describe().transpose()

In [22]:
#industry3.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/103 subimt 3 times/first review/industry_description.csv')

# 根据sustainability反馈，新增state or country level的新冠控制变量。

In [107]:
#read the office retail industry data
#read the florida covid and the U.S. covid data
office=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_4_office_covid_2018-2021.csv')
retail=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_4_retail_covid_2018-2021.csv')
industrial= pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/33_4_industry_covid_2018-2021.csv')
us_covid=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/11_US_covid_number.csv')
florida_covid=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/10_florida_covid_number.csv')

us_plus_florida3=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/12_us_plus_florida_covid.csv')

In [106]:
us_covid

,Unnamed: 0,date,date2,US_current_case,US_current_death,quarter
0,0,2020-03-31,2020-03-31,188461,4289.0,2020 Q1
1,1,2020-06-30,2020-06-30,2464860,123173.0,2020 Q2
2,2,2020-09-30,2020-09-30,4609414,79390.0,2020 Q3
3,3,2020-12-31,2020-12-31,12762066,139198.0,2020 Q4
4,4,2021-03-31,2021-03-31,10454085,205611.0,2021 Q1
5,5,2021-06-30,2021-06-30,3160878,52799.0,2021 Q2
6,6,2021-09-30,2021-09-30,9818268,93676.0,2021 Q3
7,7,2021-12-31,2021-12-31,11231325,126200.0,2021 Q4


In [72]:
florida_covid.columns=['Unnamed: 0', 'date', 'date2', 'florida_current_case', 'florida_current_death',
       'quarter']

In [74]:
#merge us and florida

us_plus_florida = pd.concat([florida_covid, us_covid], axis=1)
us_plus_florida.columns

Index(['Unnamed: 0', 'date', 'date2', 'florida_current_case',
       'florida_current_death', 'quarter', 'Unnamed: 0', 'date', 'date2',
       'US_current_case', 'US_current_death', 'quarter'],
      dtype='object')

In [75]:
us_plus_florida2=us_plus_florida[['florida_current_case',
       'florida_current_death', 'quarter','US_current_case', 'US_current_death']]

In [78]:
us_plus_florida2.columns=['florida_current_case', 'florida_current_death', 'quarter', 'quarter2',
       'US_current_case', 'US_current_death']

In [80]:
us_plus_florida3=us_plus_florida2[['florida_current_case', 'florida_current_death', 'quarter', 
       'US_current_case', 'US_current_death']]

In [81]:
us_plus_florida3

,florida_current_case,florida_current_death,quarter,US_current_case,US_current_death
0,6742,85.0,2020 Q1,188461,4289.0
1,145684,3419.0,2020 Q2,2464860,123173.0
2,554082,10812.0,2020 Q3,4609414,79390.0
3,616799,7356.0,2020 Q4,12762066,139198.0
4,734420,11752.0,2021 Q1,10454085,205611.0
5,271757,4453.0,2021 Q2,3160878,52799.0
6,1241268,17132.0,2021 Q3,9818268,93676.0
7,595640,7495.0,2021 Q4,11231325,126200.0


## 新冠数字取对数

In [94]:
us_plus_florida3['ln_florida_current_case']=''
us_plus_florida3['ln_us_current_case']=''

us_plus_florida3['ln_florida_current_death']=''
us_plus_florida3['ln_us_current_death']=''

us_plus_florida3['ln_florida_current_case']=np.log(us_plus_florida3['florida_current_case'])
us_plus_florida3['ln_us_current_case']=np.log(us_plus_florida3['US_current_case'])

us_plus_florida3['ln_florida_current_death']=np.log(us_plus_florida3['florida_current_death'])
us_plus_florida3['ln_us_current_death']=np.log(us_plus_florida3['US_current_death'])


In [96]:
us_plus_florida3.head(4)

,florida_current_case,florida_current_death,quarter,US_current_case,US_current_death,ln_florida_current_case,ln_us_current_case,ln_florida_current_death,ln_us_current_death
0,6742,85.0,2020 Q1,188461,4289.0,8.816112,12.146646,4.442651,8.363809
1,145684,3419.0,2020 Q2,2464860,123173.0,11.889195,14.717646,8.137103,11.721345
2,554082,10812.0,2020 Q3,4609414,79390.0,13.225068,15.343611,9.288412,11.282128
3,616799,7356.0,2020 Q4,12762066,139198.0,13.332298,16.361988,8.903272,11.843653


In [101]:
#us_plus_florida3.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/12_us_plus_florida_covid.csv')

## 把state and country level的新冠数字 merge 到三个data frame 中


In [108]:
office3=office.merge(us_plus_florida3,left_on='Period',right_on='quarter',how='left')
retail3=retail.merge(us_plus_florida3,left_on='Period',right_on='quarter',how='left')
industry3=industrial.merge(us_plus_florida3,left_on='Period',right_on='quarter',how='left')

In [109]:
office3.head(30)

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Unnamed: 0,florida_current_case,florida_current_death,quarter,US_current_case,US_current_death,ln_florida_current_case,ln_us_current_case,ln_florida_current_death,ln_us_current_death
0,Office,2018 Q1,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Office,2018 Q2,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Office,2018 Q3,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Office,2018 Q4,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Office,2019 Q1,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Office,2019 Q2,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Office,2019 Q3,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Office,2019 Q4,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Office,2020 Q1,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,0.0,6742.0,85.0,2020 Q1,188461.0,4289.0,8.816112,12.146646,4.442651,8.363809
9,Office,2020 Q2,All,2022 Q1,Daytona Beach - FL,19660,Office,Base Case,Metro,G41649,...,1.0,145684.0,3419.0,2020 Q2,2464860.0,123173.0,11.889195,14.717646,8.137103,11.721345


In [125]:
#fill nan with 0

office3[['ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death',
        'florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death']]=office3[['ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death','florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death',]].fillna(0)


retail3[['ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death',
        'florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death']]=retail3[['ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death','florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death',]].fillna(0)


industry3[['ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death',
        'florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death']]=industry3[['ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death','florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death',]].fillna(0)



In [126]:
industry3

,Property Class Name,Period,Slice,As Of,Geography Name,CBSA Code,Property Type,Forecast Scenario,Geography Type,Geography Code,...,Unnamed: 0,florida_current_case,florida_current_death,quarter,US_current_case,US_current_death,ln_florida_current_case,ln_us_current_case,ln_florida_current_death,ln_us_current_death
0,Industrial,2018 Q1,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,NaN,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,Industrial,2018 Q2,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,NaN,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,Industrial,2018 Q3,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,NaN,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,Industrial,2018 Q4,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,NaN,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,Industrial,2019 Q1,All,2022 Q1,Daytona Beach - FL,19660,Industrial,Base Case,Metro,G41649,...,NaN,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Industrial,2020 Q4,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,3.0,616799.0,7356.0,2020 Q4,12762066.0,139198.0,13.332298,16.361988,8.903272,11.843653
380,Industrial,2021 Q1,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,4.0,734420.0,11752.0,2021 Q1,10454085.0,205611.0,13.506836,16.162503,9.371779,12.233741
381,Industrial,2021 Q2,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,5.0,271757.0,4453.0,2021 Q2,3160878.0,52799.0,12.512664,14.966360,8.401333,10.874248
382,Industrial,2021 Q3,All,2022 Q1,The Villages - FL,45540,Industrial,Base Case,Metro,G39580,...,6.0,1241268.0,17132.0,2021 Q3,9818268.0,93676.0,14.031644,16.099755,9.748703,11.447597


In [127]:
# office3.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_6_office_covid_2018-2021_include_us_florida_covid.csv',index=0)

# retail3.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/32_6_retail_covid_2018-2021_include_us_florida_covid.csv',index=0)

# industry3.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/33_6_industry_covid_2018-2021_include_us_florida_covid.csv',index=0)

# get the correlation

In [3]:

office=pd.read_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/31_6_office_covid_2018-2021_include_us_florida_covid.csv')

In [6]:
office.columns

Index(['Property Class Name', 'Period', 'Slice', 'As Of', 'Geography Name',
       'CBSA Code', 'Property Type', 'Forecast Scenario', 'Geography Type',
       'Geography Code',
       ...
       'Unnamed: 0', 'florida_current_case', 'florida_current_death',
       'quarter', 'US_current_case', 'US_current_death',
       'ln_florida_current_case', 'ln_us_current_case',
       'ln_florida_current_death', 'ln_us_current_death'],
      dtype='object', length=108)

In [7]:
office_cor=office[['ln_cases_current', 'ln_cases_prior', 'ln_death_current','ln_death_prior',
                   'ln_florida_current_case', 'ln_us_current_case',
                   'ln_florida_current_death', 'ln_us_current_death',
                   'Total Employment','percentage_stay at home', 'percentage_facial mask']]

In [8]:
office_cor

,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior,ln_florida_current_case,ln_us_current_case,ln_florida_current_death,ln_us_current_death,Total Employment,percentage_stay at home,percentage_facial mask
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,202634.593750,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,203406.906250,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,204901.390625,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,205294.500000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,205313.906250,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
379,7.927324,7.584773,3.912023,3.970292,13.332298,16.361988,8.903272,11.843653,31842.000000,0.0,0.0
380,8.217439,7.927324,4.919981,3.912023,13.506836,16.162503,9.371779,12.233741,31496.189453,0.0,0.0
381,6.586172,8.217439,3.295837,4.919981,12.512664,14.966360,8.401333,10.874248,31540.269531,0.0,0.0
382,8.406262,6.586172,4.969813,3.295837,14.031644,16.099755,9.748703,11.447597,31965.048828,0.0,0.0


In [10]:
corrMatrix = office_cor.corr()

In [11]:
corrMatrix

,ln_cases_current,ln_cases_prior,ln_death_current,ln_death_prior,ln_florida_current_case,ln_us_current_case,ln_florida_current_death,ln_us_current_death,Total Employment,percentage_stay at home,percentage_facial mask
ln_cases_current,1.000000,0.929994,0.973075,0.876552,0.971920,0.964901,0.975700,0.964760,0.106457,0.279194,0.343981
ln_cases_prior,0.929994,1.000000,0.956173,0.972408,0.899022,0.882437,0.919332,0.882350,0.092816,0.216451,0.315155
ln_death_current,0.973075,0.956173,1.000000,0.913185,0.927078,0.910730,0.945907,0.913077,0.137557,0.267302,0.347986
ln_death_prior,0.876552,0.972408,0.913185,1.000000,0.834431,0.817935,0.851381,0.810495,0.129157,0.159154,0.280199
ln_florida_current_case,0.971920,0.899022,0.927078,0.834431,1.000000,0.998099,0.995793,0.997082,-0.010804,0.233235,0.298111
ln_us_current_case,0.964901,0.882437,0.910730,0.817935,0.998099,1.000000,0.988819,0.998540,-0.010597,0.234560,0.294056
ln_florida_current_death,0.975700,0.919332,0.945907,0.851381,0.995793,0.988819,1.000000,0.990260,-0.011593,0.238096,0.307075
ln_us_current_death,0.964760,0.882350,0.913077,0.810495,0.997082,0.998540,0.990260,1.000000,-0.011551,0.245796,0.298278
Total Employment,0.106457,0.092816,0.137557,0.129157,-0.010804,-0.010597,-0.011593,-0.011551,1.000000,0.182648,0.161612
percentage_stay at home,0.279194,0.216451,0.267302,0.159154,0.233235,0.234560,0.238096,0.245796,0.182648,1.000000,0.651636


In [13]:
#corrMatrix.to_csv('/Users/yoga/Dropbox/1 新冠和商业地产/101 CRE second time 202205/new_correlation_For_sustainability.csv')